<a href="https://www.kaggle.com/code/oswind/stockchat-agents-edition?scriptVersionId=283173476" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Environment Setup

In [ ]:
# Setup the notebook based on running environment.
import os
# Optional: Enable telemetry in browser_use and chromadb.
os.environ["ANONYMIZED_TELEMETRY"] = "false"
# Check for kaggle environment.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    # Kaggle Run: update the system.
    !pip uninstall -qqy google-ai-generativelanguage pydrive2 tensorflow tensorflow-decision-forests cryptography pyOpenSSL langchain langchain-core nltk ray click google-generativeai google-cloud-translate datasets cesium bigframes plotnine mlxtend fastai spacy thinc google-colab gcsfs jupyter-kernel-gateway nltk preprocessing
    !pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    !pip install -qU langchain-community langchain-text-splitters wikipedia lmnr[all] google-adk google-adk[eval] google-cloud-translate
    from kaggle_secrets import UserSecretsClient # type: ignore
    from jupyter_server.serverapp import list_running_servers # type: ignore
else:
    # Mock the kaggle secrets client.
    class UserSecretsClient:
        @classmethod
        def set_secret(cls, id: str, value: str):
            os.environ[id] = value
        @classmethod
        def get_secret(cls, id: str):
            try:
                return os.environ[id]
            except KeyError as e:
                print(f"KeyError: authentication token for {id} is undefined")
    # Local Run: update the venv.
    %pip install -qU posthog\<6.0.0 google-genai==1.50.0 chromadb==0.6.3 opentelemetry-proto==1.37.0
    %pip install -qU langchain-community langchain-text-splitters wikipedia pandas google-api-core "lmnr[all]" browser-use ollama google-adk "google-adk[eval]"
    from browser_use import Agent as BrowserAgent

import ast, chromadb, json, logging, pandas, platform, pytz, re, requests, sys, threading, time, warnings, wikipedia
from bs4 import Tag
from chromadb import Documents, Embeddings
from datetime import datetime, timedelta
from dateutil.parser import parse
from enum import Enum
from google.adk.apps.app import App
from google.adk.sessions import InMemorySessionService, BaseSessionService as SessionService, Session
from google.adk.runners import Runner, Event
from google import genai
from google.api_core import retry, exceptions
from google.genai.models import Models
from google.genai import types, errors
from IPython.display import Markdown, display, HTML
from langchain_community.document_loaders.csv_loader import CSVLoader
from langchain_text_splitters.html import HTMLSemanticPreservingSplitter
from langchain_text_splitters.json import RecursiveJsonSplitter
from lmnr import Laminar
from math import inf
from pydantic import BaseModel, field_validator
from threading import Timer
from tqdm import tqdm
from typing import Optional, Callable, NewType, NamedTuple
from wikipedia.exceptions import DisambiguationError, PageError

In [2]:
# Prepare the Gemini api for use.
# Setup a retry helper for generation not run through the below api-helper.
is_retriable = lambda e: (isinstance(e, errors.APIError) and e.code in {429, 503, 500})
Models.generate_content = retry.Retry(predicate=is_retriable)(Models.generate_content)
Models.embed_content = retry.Retry(predicate=is_retriable)(Models.embed_content)

# Activate Laminar auto-instrumentation.
try:
    Laminar.initialize(project_api_key=UserSecretsClient().get_secret("LMNR_PROJECT_API_KEY"))
except:
    print("Skipping Laminar.initialize()")

class GeminiModel:
    def __init__(self, rpm: list, tpm: list, rpd: list):
        self.rpm = rpm # requests per minute
        self.tpm = tpm # tokens per minute in millions
        self.rpd = rpd # requests per day
        self.err = [0,0] # validation, api_related

# A python api-helper with model fail-over/chaining/retry support.
GeminiEmbedFunction = NewType("GeminiEmbedFunction", None) # forward-decl
class Api:
    gen_limit_in = 1048576
    emb_limit_in = 2048
    gen_model = {
        "gemini-2.5-flash": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # stable: 10 RPM/250K TPM/250 RPD
        "gemini-2.5-flash-preview-09-2025": GeminiModel([10,1000,2000,10000],[.25,1,3,8],[250,10000,100000,inf]), # exp: 10 RPM/250K TPM/250 RPD
        "gemini-2.0-flash-exp": GeminiModel([10,10,10,10],[.25,.25,.25,.25],[200,500,500,500]), # latest w/thinking: 10 RPM/250K TPM/200 RPD
        "gemini-2.0-flash": GeminiModel([15,2000,10000,30000],[1,4,10,30],[200,inf,inf,inf]), # stable wo/thinking: 15 RPM/1M TPM/200 RPD
        "gemini-2.5-flash-lite": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # stable: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-flash-lite-preview-09-2025": GeminiModel([15,4000,10000,30000],[.25,4,10,30],[1000,inf,inf,inf]), # exp: 15 RPM/250K TPM/1K RPD
        "gemini-2.5-pro": GeminiModel([5,150,1000,2000],[.125,2,5,8],[100,10000,50000,inf]), # stable: 5 RPM/250K TPM/100 RPD
    }
    gen_local = ["gemma3n:e4b","gemma3:12b-it-qat"]
    default_local = 0
    default_model = []
    embed_model = "gemini-embedding-001", GeminiModel([100,3000,5000,10000],[.03,1,5,10],[1000,inf,inf,inf]) # stable: 100 RPM/30K TPM/1000 RPD/100 per batch
    embed_local = False
    error_total = 0
    min_rpm = 3
    min_tpm = 40000
    dt_between = 2.0
    errored = False
    running = False
    dt_err = 45.0
    dt_rpm = 60.0

    @classmethod
    def get(cls, url: str):
        # Create a header matching the OS' tcp-stack fingerprint.
        system_ua = None
        match platform.system():
            case 'Linux':
                system_ua = 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
            case 'Darwin':
                system_ua = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 15_7_2) AppleWebKit/605.1.15 (KHTML, like Gecko) Version/26.0 Safari/605.1.15'
            case 'Windows':
                system_ua = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/142.0.0.0 Safari/537.36'
        try:
            request = requests.get(url, headers={'User-Agent': system_ua})
            if request.status_code != requests.codes.ok:
                print(f"Api.get() returned status {request.status_code}")
            return request.text
        except Exception as e:
            raise e

    class Limit(Enum):
        FREE = 0
        TIER_1 = 1
        TIER_2 = 2
        TIER_3 = 3
    
    class Model(Enum):
        GEN = 1
        EMB = 2
        LOC = 3

    class Const(Enum):
        STOP = "I don't know."
        METRIC_BATCH = 20
        SERIES_BATCH = 40
        EMBED_BATCH = 100
        CHUNK_MAX = 1500

        @classmethod
        def Stop(cls):
            return cls.STOP.value

        @classmethod
        def MetricBatch(cls):
            return cls.METRIC_BATCH.value

        @classmethod
        def SeriesBatch(cls):
            return cls.SERIES_BATCH.value

        @classmethod
        def EmbedBatch(cls):
            return cls.EMBED_BATCH.value

        @classmethod
        def ChunkMax(cls):
            return cls.CHUNK_MAX.value
    
    class Env(NamedTuple): # Make init args immutable.
        CLIENT: genai.Client
        API_LIMIT: int
        GEN_DEFAULT: str

    def __init__(self, with_limit: Limit | int, default_model: str):
        if default_model in self.gen_model.keys():
            self.write_lock = threading.RLock()
            try:
                if isinstance(with_limit, int) and with_limit in [id.value for id in Api.Limit]:
                    limit = with_limit
                else:
                    limit = with_limit.value
            except Exception as e:
                print(f"Api.__init__: {with_limit} is not a valid limit")
            else:
                self.args = Api.Env(
                    genai.Client(api_key=UserSecretsClient().get_secret("GOOGLE_API_KEY")),
                    limit, default_model)
            self.m_id = list(self.gen_model.keys()).index(default_model)
            self.default_model.append(default_model)
            self.update_quota()
            self.s_embed = GeminiEmbedFunction(self.args.CLIENT, semantic_mode = True) # type: ignore
            logging.getLogger("google_genai").setLevel(logging.WARNING) # suppress info on generate
        else:
            print(f"Api.__init__: {default_model} not found in gen_model.keys()")
        

    def __call__(self, model: Model) -> str:
        if model == self.Model.GEN:
            return "models/" + list(self.gen_model.keys())[self.m_id]
        elif model == self.Model.LOC:
            return self.gen_local[self.default_local]
        else:
            return "models/" + self.embed_model[0] if not self.embed_local else "embeddinggemma:latest"

    def push_default_model(self, model_id: str):
        if model_id in self.gen_model.keys():
            self.write_lock.acquire()
            self.stop_running()
            self.default_model.append(model_id)
            self.m_id = list(self.gen_model.keys()).index(model_id)
            self.write_lock.release()
        else:
            print(f"{model_id} not found in gen_model.keys()")

    def pop_default_model(self):
        if len(self.default_model) > 1:
            self.write_lock.acquire()
            self.stop_running()
            self.default_model.pop(-1)
            self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
            self.write_lock.release()

    def retriable(self, retry_fn: Callable, *args, **kwargs):
        tries = 3*len(self.gen_model.keys())
        for attempt in range(tries):
            try:
                self.write_lock.acquire()
                token_use = self.token_count(kwargs["contents"])
                if self.gen_rpm > self.min_rpm and token_use <= self.token_quota and self.token_quota > self.min_tpm:
                    self.token_quota -= token_use
                    self.gen_rpm -= 1
                else:
                    self.on_error(kwargs)
                if not self.running and not self.errored:
                    self.rpm_timer = Timer(self.dt_rpm, self.refill_rpm)
                    self.rpm_timer.start()
                    self.running = True
                return retry_fn(*args, **kwargs)
            except (errors.APIError, exceptions.RetryError) as api_error:
                if isinstance(api_error, errors.APIError):
                    is_retry = api_error.code in {429, 503, 500, 400} # code 400 when TPM exceeded
                    if api_error.code == 400:
                        print(f"retriable.api_error: token limit exceeded ({token_use})")
                    else:
                        print(f"retriable.api_error({api_error.code}): {str(api_error)}")
                    if not is_retry or attempt == tries:
                        raise api_error
                self.on_error(kwargs)
            except Exception as e:
                print(f"retriable.exception: {str(e)}")
                self.on_error(kwargs)
            finally:
                self.write_lock.release()

    def on_error(self, kwargs):
        self.generation_fail()
        kwargs["model"] = self(Api.Model.GEN)
        time.sleep(self.dt_between)

    def stop_running(self):
        if self.running:
            self.rpm_timer.cancel()
            self.running = False

    def validation_fail(self):
        list(self.gen_model.values())[self.m_id].err[0] += 1
        self.error_total += 1

    def generation_fail(self):
        self.stop_running()
        self.save_error()
        self.next_model()
        print("Api.generation_fail.next_model: model is now", list(self.gen_model.keys())[self.m_id])
        if not self.errored:
            self.error_timer = Timer(self.dt_err, self.zero_error)
            self.error_timer.start()
            self.errored = True

    def save_error(self):
        list(self.gen_model.values())[self.m_id].err[1] += 1
        self.error_total += 1

    def next_model(self):
        self.m_id = (self.m_id+1)%len(self.gen_model.keys())
        self.update_quota()

    def refill_rpm(self):
        self.running = False
        self.update_quota()
        print("Api.refill_rpm", self.gen_rpm)

    def zero_error(self):
        self.errored = False
        self.m_id = list(self.gen_model.keys()).index(self.default_model[-1])
        self.update_quota()
        print("Api.zero_error: model is now", list(self.gen_model.keys())[self.m_id])

    def update_quota(self):
        self.gen_rpm = list(self.gen_model.values())[self.m_id].rpm[self.args.API_LIMIT]
        self.token_quota = list(self.gen_model.values())[self.m_id].tpm[self.args.API_LIMIT]*1_000_000

    def token_count(self, expr: str | list):
        count = self.args.CLIENT.models.count_tokens(
            model=self(Api.Model.GEN),
            contents=json.dumps(expr) if isinstance(expr, str) else str(expr))
        return count.total_tokens

    def errors(self):
        errors = {"total": self.error_total, "by_model": {}}
        for m_code, m in self.gen_model.items():
            errors["by_model"].update({
                m_code: {
                    "api_related": m.err[1],
                    "validation": m.err[0]
                }})
        return errors

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def similarity(self, content: list):
        return self.s_embed.sts(content) # type: ignore

Skipping Laminar.initialize()


In [3]:
# Define the embedding function.
api = NewType("api", Api) # type: ignore (forward-decl)
class GeminiEmbedFunction:
    document_mode = True  # Generate embeddings for documents (T,F), or queries (F,F).
    semantic_mode = False # Semantic text similarity mode is exclusive (F,T).
    
    def __init__(self, genai_client, semantic_mode: bool = False):
        self.client = genai_client
        if semantic_mode:
            self.document_mode = False
            self.semantic_mode = True

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __embed__(self, input: Documents) -> Embeddings:
        if self.document_mode:
            embedding_task = "retrieval_document"
        elif not self.document_mode and not self.semantic_mode:
            embedding_task = "retrieval_query"
        elif not self.document_mode and self.semantic_mode:
            embedding_task = "semantic_similarity"
        partial = self.client.models.embed_content(
            model=api(Api.Model.EMB),
            contents=input,
            config=types.EmbedContentConfig(task_type=embedding_task)) # type: ignore
        return [e.values for e in partial.embeddings]
    
    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def __call__(self, input: Documents) -> Embeddings:
        try:
            response = []
            for i in range(0, len(input), Api.Const.EmbedBatch()):  # Gemini max-batch-size is 100.
                response += self.__embed__(input[i:i + Api.Const.EmbedBatch()])
            return response
        except Exception as e:
            print(f"caught exception of type {type(e)}\n{e}")
            raise e

    def sts(self, content: list) -> float:
        df = pandas.DataFrame(self(content), index=content)
        score = df @ df.T
        return score.iloc[0].iloc[1]

## Set Gemini API Limit

In [4]:
# Instantiate the api-helper with usage limit => FREE.
# Optional: Set limit here to one of [FREE,TIER_1,TIER_2,TIER_3]
api = Api(with_limit=Api.Limit.FREE, default_model="gemini-2.5-flash")
# Export api environment for agent.
os.environ["API_LIMIT"]=str(api.args.API_LIMIT)
os.environ["GEN_DEFAULT"]=api.args.GEN_DEFAULT
# Cleanup old vector_db instances.
!rm -rf vector_db

# __StockChat: Agents Edition__

It was during Kaggle's 5-day Generative AI course in 2025 that StockChat first existed as a simple search-connected LLM. There were two observations from that initial build. First being the need for a real-time source of grounding truth. Even with google-search data was more often incomplete. The second observation, which still exists today, is the tendency toward hallucinations in finance data. Ticker symbols can imitate the name of another company, and it's also possible for the LLM to confuse a company name for a wrong symbol. This happens even when the context of the question matches the immediate discussion history and should be self-evident.

```python
response = chat.send_message('''What is MGM Studio's stock ticker symbol?''')
Markdown(response.text)
```

```
... (possibly useful content, often not)

It is important not to confuse MGM Studios with MGM Resorts International, which is a separate, publicly traded hospitality and casino entertainment company with the stock ticker symbol MGM on the New York Stock Exchange (NYSE).
```

Gemini is naturally chatty in a helpful way and this sometimes causes it to go off-topic. The inclusion of off-topic discussion requires that all output from the LLM be checked for topic deviations. Otherwise a backing RAG may store incorrect truths. It became a trade-off between restraining gemini output, and it's usefulness, versus unrestrained with the hallucination caveat. So google-search was not the solution, and actually it was kind-of off-putting as a source of finance chat. Thus StockChat transformed into a huge monolithic agent with access to multiple finance api's, and wikipedia/search to back it up.

```python
send_message("What is MGM Studio's stock symbol?")
```

```
MGM Studios (Metro-Goldwyn-Mayer Studios, Inc.) is a wholly-owned subsidiary of Amazon and is not publicly traded, so it does not have its own stock symbol.

The stock symbol for its parent company, Amazon, is AMZN.
```

While big and capable, StockChat (SC1) became limited by it's single agent design.
- There’s no parallelism or asynchronous operation because parallel function-calling is agent-wide. Some of the functions may have unmet dependencies when run parallel (by an LLM). In other cases the degree of parallelism is determined by whether you have paid for finance api access. As I’m building a toy I wanted to keep free-tier as an option. SC1 is essentially an LLM-guided loop of serial operations, along with a single rest api request at a time. It makes SC1 stable at the cost of performance and responsiveness.
- The lack of context management means SC1 can handle months worth of pre-scored news data. The context is shared by all tools while answering a user question. Meaning tools which are called last are limited by, sometimes large, unrelated context generated earlier.
- There's a single vector store with all acquired data, requiring metadata management to compensate. The store is optimized for document search, and only uses similarity optimizations for temporary vectors.
- SC1 has no facility to determine user interest. It's a giant cache of previously searched finance data.
- There's no systematic evaluation except to run baseline queries.

With these issues in mind my goal during Kaggle's 5-day Agents course was to apply Google's agentic framework to free SC1 from these limitations.
- SC2 uses async runners while maintaining minimal thread synchronization on shared data.
- LLM-assisted context compaction runs at regular intervals.
- All the sub-tools have their own vector stores.
- A memory tool stores long-term memories with semantic meaning and date of creation.
- A user profile expert extracts user attributes and preferences for long-term memory.
- Session state keys share points-of-view without the entirety of each agent's context.
- A summary writing expert ensures large generations aren't blemished by erratic formatting.
- The ADK CLI is used to run an evaluation suite with LLM-as-judge.

## Setup working directory

On Kaggle, the working directory for `google.adk.runners.Runner` differs from notebook location. To work around this I use git with sparse-checkout to pull in SC2's updated source. Then I setup the Kaggle runner environment and define the async runner.

In [10]:
# Setup working directory on Kaggle.
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    if not os.path.isdir("sc2/"):
        !git init -b main
        !git remote add origin https://github.com/lol-dungeonmaster/kaggle-agents-2025.git
        !git config core.sparseCheckout true
        !echo "sc2/" >> .git/info/sparse-checkout
        !git pull origin main
        for api_key in ["GOOGLE_API_KEY","POLYGON_API_KEY","FINNHUB_API_KEY"]:
            env_key = UserSecretsClient().get_secret(api_key)
            !echo "$api_key=$env_key" >> sc2/.env # from .venv on local runs
            os.environ[api_key] = UserSecretsClient().get_secret(api_key) # from .venv on local runs

In [11]:
# Define async runner and helper functions.
from sc2.agent import app
from sc2.src import log
# Logger access not possible on Kaggle to prevent hiding errors.
# - The StderrToLog wrapper will not work as kaggle-docker makes the file-descriptor constant.
# - Redirect basic logger output on Kaggle using print().
if os.getenv("KAGGLE_KERNEL_RUN_TYPE"):
    log_info = print
    log_warn = print
    log_err = print
else:
    log_info = log.info
    log_warn = log.warning
    log_err = log.error

# Display the user query and response after the response is complete.
async def on_event(e: Event, q: str):
    try:
        response = e.content.parts[0].text
        if response and response != "None":
            log_info(f"USER  > {q}")
            log_info(f"MODEL > {response}\n")
    except Exception as err:
        log_err(f"on_event.exception: {str(err)}")

# Run an App with the provided BaseSessionService and user queries list.
async def run_queries(app: App, sessions: SessionService, queries: list[str],
                      session_id: str = "default", user_id: str = "default"):
    runner = Runner(app=app, session_service=sessions)
    try:
        session = await sessions.create_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )
    except:
        session = await sessions.get_session(
            app_name=runner.app_name, user_id=user_id, session_id=session_id
        )
    finally:
        log_info(f"### Agent session: (uid={user_id}) {session_id}\n")
        for query in queries:
            await try_run(runner, session, user_id, query)

# Launch a runner with the provided session and user_id then respond to query.
# - retries on exceptions TypeError, KeyError, IndexError
async def try_run(runner: Runner, session: Session, user_id: str, query: str):
    try:
        q = types.Content(role="user", parts=[types.Part(text=query)])
        async for response in runner.run_async(
            user_id=user_id, session_id=session.id, new_message=q
        ): await on_event(response, query)
    except Exception as e:
        q_id = " ".join(query.split()[:4])
        if type(e) in [TypeError, KeyError, IndexError]:
            log_warn(f"try_run.run_async (q={q_id}): retrying, generated {type(e).__name__}\n")
            time.sleep(15.0)
            await try_run(runner, session, user_id, query)
        else:
            log_warn(f"try_run.run_async (q={q_id}): {type(e).__name__} - {str(e)}\n")

# Check for compaction events in the provided BaseSessionService.
# - optionally also show the llm compacted output.
async def check_compaction(sessions: SessionService, session_id: str = "default", 
                           user_id: str = "default", show_llm: bool = False):
    n = 0
    for e in (await sessions.get_session(
        app_name=app.name,
        user_id=user_id,
        session_id=session_id,
    )).events:
        if e.actions and (llm_out := e.actions.compaction):
            n += 1
            if show_llm:
                log_info(f"check_compaction.show_llm: {llm_out.compacted_content.parts[0].text}\n")
    log_info(f"check_compaction: found ({n}) compaction event\n")

WARNING  [root] KeyError: authentication token for LMNR_PROJECT_API_KEY is undefined
INFO     [root] Skipping Laminar.initialize()
INFO     [root] sc2.__init__: the api-helper is ready
Generate document embedding: 100%|##########| 1/1 [00:04<00:00,  4.13s/it]
INFO     [root] sc2.__init__: RestGroundingTool is ready
INFO     [root] sc2.__init__: SearchGroundingTool is ready
INFO     [root] sc2.__init__: WikiGroundingTool is ready
INFO     [root] sc2.__init__: MemoryService is ready
WARNING  [root] [EXPERIMENTAL] ReflectAndRetryToolPlugin: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  plugins=[ReflectAndRetryToolPlugin(max_retries=1)],
WARNING  [root] [EXPERIMENTAL] EventsCompactionConfig: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  events_compaction_config=EventsCompactionConfig(


## Test the Runner

The two initial questions are used to test the agents self-awareness of tools. This was particularly problematic for the parallel `fncall_pipeline`. The goal is to have a parallel operating planner and executor of function calls. The function tool definition are tricky to access reliably when nested inside workflow agents like the ParallelAgent. In the end I exposed the planner and it's containing pipeline, then told Gemini where to look.

Another objective is to make SC2 a more capable assistant in addition to removing existing limits. To that end I also added a Terminology expert to make use of the built-in google-search. Meanwhile a user profile expert dynamically extracts preferences and user attributes. These two types of data are stored in long-term memory.

In [12]:
# Create a session service and run some test queries.
s_svc = InMemorySessionService()

await run_queries(
    app=app, sessions=s_svc,
    queries=[
        "What tools do you know how to use?",
        "Tell me what functions `fncall_pipeline` knows by checking `sc2_fnplan`.",
        "What is a short trade?",
        "What is gambler's ruin?",
        "My local advisor is SC at JPMorgan Chase, 212-736-2001",
        "I live in Brooklyn, New York."])

INFO     [root] ### Agent session: (uid=default) default

INFO     [root] USER  > What tools do you know how to use?
INFO     [root] MODEL > I can use the following tools:

*   `sc2_memory`: An expert writer of long-term memories.
*   `sc2_prefs`: An expert profile analyst in the field of finance, money, and stock markets.
*   `fncall_pipeline`: A function caller with functions defined in sub-agent `sc2_fnplan`.
*   `sc2_fnplan`: A highly intelligent FunctionTool call planner.
*   `sc2_terms`: An expert terminologist in the field of finance, money, and stock markets.
*   `sc2_summary`: An expert proof-reader and writer that knows HTML, JSON and Markdown.

INFO     [root] USER  > Tell me what functions `fncall_pipeline` knows by checking `sc2_fnplan`.
INFO     [root] MODEL > `fncall_pipeline` knows the following functions:

*   `get_symbol_1`
*   `get_symbols_1`
*   `get_name_1`
*   `get_symbol_quote_1`
*   `get_market_status_1`
*   `get_market_session_1`
*   `get_company_peers_1`
*   `

## Test Long-term Memory

Testing long-term memory is as easy as creating a new `BaseSessionService`. As this Memory is a custom implementation it must be specified as a tool during user query. That's because the objective of knowing user profile data is secondary to the goal of being a capable finance assistant. A new root agent which checks Memory and question context before delegation can fix this (if needed) in future refinements.

In [13]:
# Use long-term memory from a new session.
s_svc2 = InMemorySessionService()

await run_queries(
    app=app, sessions=s_svc2,
    queries=[
        "Check memory for where I live.",
        "Check memory for my local advisor SCs phone number."])

INFO     [root] ### Agent session: (uid=default) default

INFO     [root] Api.refill_rpm 10
INFO     [root] USER  > Check memory for where I live.
INFO     [root] MODEL > You live in Brooklyn, New York.

INFO     [root] USER  > Check memory for my local advisor SCs phone number.
INFO     [root] MODEL > SC's phone number is 212-736-2001.



## Check for compaction

One of the features of SC2 that I’m looking forward to working with more is the LLM-assisted context compaction. In this implementation I’ve opted for zero-overlap to avoid re-summarizing past events. At this point no events are dropped from the context. The LLM is known to become confused with statement repetition, so let’s avoid that complication. A delightful feature of LLM-compaction is the use of an LLM-as-judge to assess the context's quality with impartiality. It’ll note neat things for you like when the tools fail completely or when parts of a user query remain unanswered. That leaves further room for improvements in context management. We can prune events that describe failures or reflect and correct on them.

In [14]:
# Display context compaction output.
await check_compaction(s_svc, show_llm=True)

INFO     [root] check_compaction.show_llm: The user initiated the conversation by asking about the AI's available tools. The AI listed its tools, including `sc2_memory`, `sc2_prefs`, `fncall_pipeline`, `sc2_fnplan`, `sc2_terms`, and `sc2_summary`, along with their brief descriptions.

Next, the user asked the AI to specify the functions available through `fncall_pipeline` by checking `sc2_fnplan`. The AI responded by listing 20 specific functions, predominantly related to financial data retrieval (e.g., `get_symbol_1`, `get_market_status_1`, `get_financials_1`, `get_news_with_sentiment_2`).

Finally, the user asked for a definition of "short trade." The AI provided a clear explanation, describing it as a speculative strategy where an investor sells borrowed securities with the expectation of repurchasing them at a lower price to profit from a price decrease.

**Key information and decisions made:**
*   The AI disclosed its capabilities in terms of tools and specific functions.
*   The 

## Evaluation by CLI

In SC1 evaluation didn't happen systematically. As you can see from the appendix evaluation consists of manually checking the model output, or baseline. In leveraging the ADK CLI, SC2 gains an LLM-as-judge to systematically evaluate assistant output. A rubric is applied to check response quality and related tool use. Then a hallucination test is performed to ensure the agent has stayed on-topic.

In [15]:
!adk eval sc2 sc2/eval/test_cases.json --config_file_path=sc2/eval/config.json --print_detailed_results

/home/sysop/Documents/kaggle-agents-2025/.venv/lib/python3.12/site-packages/google/adk/evaluation/metric_evaluator_registry.py:90: UserWarning: [EXPERIMENTAL] MetricEvaluatorRegistry: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  metric_evaluator_registry = MetricEvaluatorRegistry()
/home/sysop/Documents/kaggle-agents-2025/.venv/lib/python3.12/site-packages/google/adk/evaluation/local_eval_service.py:80: UserWarning: [EXPERIMENTAL] UserSimulatorProvider: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  user_simulator_provider: UserSimulatorProvider = UserSimulatorProvider(),
Using evaluation criteria: criteria={'rubric_based_final_response_quality_v1': BaseCriterion(threshold=0.8, judge_model_options={'judge_model': 'gemini-2.5-flash', 'num_samples': 1}, rubrics=[{'rubric_id': 'conciseness', 'rubric_

## Conclusion

In applying Google's ADK to SC1, the result is a more capable SC2 which is ready to scale beyond it's first edition roots. Unresolved issues from SC1 remain. The introduced improvements will enable large work loads, like a stack of news requiring analysis, or background processes to drive self-improvement. This will result in a faster and more responsive user experience. SC2 will benefit when local models are employed, with fewer and more specific tasks delegated to them. With the addition of agentic capabilities StockChat has room to grow again.

__I hope you'll stick around to see how far the project gets! Thanks for taking the time to check out my notebook!__

# __Appendix__

## Gemini Baseline Check

In [5]:
# This is an accurate retelling of events. 
config_with_search = types.GenerateContentConfig(
    tools=[types.Tool(google_search=types.GoogleSearch())],
    temperature=0.0
)

chat = api.args.CLIENT.chats.create(
    model=api(Api.Model.GEN),
    config=config_with_search,
    history=[]) # Ignoring the part about dark elves, and tengwar.

response = chat.send_message('Do you know anything about the stock market?')
Markdown(response.text)

The stock market is a global network of exchanges and over-the-counter (OTC) marketplaces where investors buy and sell shares of publicly traded companies. These shares, also known as equities, represent fractional ownership in a company.

**Purpose of the Stock Market**
The stock market serves two primary functions:
*   **For Companies:** It allows companies to raise capital (money) by issuing shares to the public through a process called an Initial Public Offering (IPO). This capital can then be used to fund and expand their businesses.
*   **For Investors:** It provides individuals and institutions with an opportunity to invest in businesses and potentially grow their wealth over time. Investors can profit through dividends (a share of the company's profits) or by selling their shares at a higher price than they bought them for (capital gains).

**How the Stock Market Works**
The stock market operates through two main types of markets:
*   **Primary Market:** This is where new stocks are first issued. When a private company decides to go public, it lists its shares on an exchange through an IPO, selling them directly to investors to raise capital.
*   **Secondary Market:** After the initial issuance, these shares are traded among investors on stock exchanges (like the New York Stock Exchange or Nasdaq). The company is not directly involved in these subsequent transactions.

Stock prices are primarily determined by the forces of supply and demand. If more investors want to buy a stock than sell it, the price tends to rise, and vice versa. This process is known as price discovery.

**Key Components and Participants**
*   **Stock Exchanges:** These are organized and regulated platforms (often virtual) where stocks and other securities are bought and sold.
*   **Brokers:** These are intermediaries who execute buy and sell orders on behalf of investors.
*   **Investors and Traders:** Participants range from individual retail investors to large institutional investors like pension funds, mutual funds, insurance companies, and hedge funds.

**Types of Investments in the Stock Market**
Beyond just common stocks, the stock market offers various investment instruments:
*   **Stocks (Equities):**
    *   **Common Stock:** Represents partial ownership, gives voting rights on corporate decisions, and offers potential for higher returns, but also higher risk.
    *   **Preferred Stock:** Typically offers fixed dividend payments before common stockholders receive theirs, but usually does not come with voting rights.
    *   **Categorized by Market Capitalization:** Large-cap (companies with market capitalization of $10 billion or more, generally stable), Mid-cap ($2 billion to $10 billion), and Small-cap (less than $2 billion, higher growth potential but riskier).
    *   **Categorized by Investment Style:** Growth stocks (companies with strong potential for rapid growth), Value stocks (undervalued companies that investors believe will rise in price), and Income stocks (companies that pay regular, often higher-than-average, dividends).
*   **Bonds:** These are debt securities where investors essentially lend money to governments or corporations for a set period, receiving regular interest payments.
*   **Mutual Funds:** These pool money from many investors to invest in a diversified portfolio of stocks, bonds, or other securities, managed by a professional fund manager.
*   **Exchange-Traded Funds (ETFs):** Similar to mutual funds, but they trade like individual stocks on exchanges throughout the day.
*   **Derivatives:** Complex financial instruments whose value is derived from an underlying asset, such as stocks. These are generally considered high-risk and not recommended for beginners.

**Factors Influencing Stock Prices**
Stock market movements are influenced by a variety of factors, including company performance (earnings reports, product launches), macroeconomic indicators (interest rates, inflation, GDP growth), political events, international trade policies, and overall investor sentiment.

In [6]:
response = chat.send_message('I have an interest in AMZN stock')
Markdown(response.text)

Amazon.com Inc. (NASDAQ: AMZN) is a widely followed stock, known for its dominant presence in e-commerce, cloud computing (Amazon Web Services - AWS), and digital advertising.

Here's a snapshot of AMZN stock as of December 1, 2025:

**Current Price and Performance:**
*   The current price of AMZN stock is approximately $233.22 USD.
*   It has seen a 1.77% increase in the past 24 hours.
*   Over the past week, AMZN stock has risen by 7.80%, and over the last year, it has shown a 12.68% increase.
*   The stock's 52-week high is $258.60 and its 52-week low is $161.38.

**Key Financials and Metrics:**
*   Amazon's market capitalization is approximately $2.49 trillion.
*   The company's Price-to-Earnings (P/E) ratio is around 32.95.
*   Amazon does not currently pay dividends to shareholders, as it reinvests earnings into growth areas.
*   AMZN stock has a beta coefficient of 1.41, indicating its volatility.

**Analyst Outlook:**
*   The overall consensus from 34 analysts over the last three months is a "Strong Buy" for AMZN.
*   The average price target for Amazon.com Inc. is around $298.13, with a maximum estimate of $360.00 and a minimum estimate of $250.00. Oppenheimer recently raised its price target to $305.00 from $290.00, maintaining an "Outperform" rating.

**Business Segments and Growth Drivers:**
*   Amazon's revenue primarily comes from its retail operations (approximately 74%), followed by Amazon Web Services (AWS) (17%), and advertising services (9%).
*   AWS is a significant growth driver, with its revenue growing by 20.2% year-over-year in the third quarter of 2025 and a reported $200 billion backlog.
*   The company continues to expand in areas such as artificial intelligence (AI), cloud computing, global e-commerce, streaming, and logistics automation.

**Recent News and Influencing Factors:**
*   Amazon Web Services recently launched a new multicloud service to enhance data movement with Google Cloud, following an outage in October that caused significant losses for U.S. companies.
*   Amazon is participating in a $400 billion investment in AI by 2025, which is expected to significantly impact the U.S. economy.
*   AI shopping tools, including Amazon's Rufus, are projected to boost traffic to U.S. retail sites by 670% this holiday season compared to last year.
*   Oppenheimer's analysis suggests significant upside potential for AWS through 2027, with plans to double its capacity.
*   Amazon's stock price can be influenced by quarterly earnings, changes in cloud computing demand, consumer spending trends, new business initiatives, and broader economic conditions like inflation and interest rates.

**Historical Context:**
*   AMZN reached its all-time high on November 2, 2025, with a price of $258.60 USD.
*   The company conducted a 20-for-1 stock split in June 2022 to make shares more accessible to retail investors.

In [7]:
response = chat.send_message('''Tell me about AMZN current share price, short-term trends, and bullish versus bearish predictions''')
Markdown(response.text)

Amazon.com Inc. (NASDAQ: AMZN) is currently trading at approximately $233.22 USD as of December 1, 2025. The stock has shown an increase of 1.77% in the past 24 hours, a 7.80% rise over the last week, and a 0.67% increase over the past month. Over the last year, AMZN has seen a 12.68% increase. The stock's 52-week high is $258.60, and its 52-week low is $161.38.

**Short-Term Trends**

In the short term, AMZN stock is exhibiting signs of strength. It has risen by 7.39% since a pivot bottom point on Thursday, November 20, 2025, with further increases indicated until a new top pivot is found. The stock holds buy signals from both short and long-term Moving Averages, suggesting a positive forecast. Technical analysis indicates that AMZN is breaking out above a horizontal support zone and shows enough bullish momentum for a break-in above its ascending Fibonacci Retracement Fan. The Bull Bear Power Indicator has also turned bullish with an ascending trendline, and average bullish trading volumes have been higher than bearish trading volumes over the past week.

However, some short-term forecasts suggest potential minor fluctuations. One prediction indicates that the value of AMZN shares could drop by -2.60% to $227.12 per share by December 30, 2025. Another short-term forecast suggests a slight decrease to $230.68 by December 2, 2025, and $229.65 by December 3, 2025, before potentially rising to $233.80 by December 5, 2025. Despite these minor predicted dips, the overall sentiment based on technical indicators is bullish.

**Bullish Versus Bearish Predictions**

The overwhelming sentiment from analysts regarding AMZN stock is bullish. A consensus of 43 analysts over the last three months rates AMZN as a "Strong Buy," with 41 assigning a "Buy" rating and two a "Hold" rating. The average price target from 44 Wall Street analysts for the next 12 months is around $295.23, representing a potential upside of 28.83% from the current price, with a high forecast of $340.00 and a low forecast of $250.00. Other analyst targets range from an average of $282.48 to $298.13, all suggesting significant upside potential.

Key drivers for these bullish predictions include:
*   **Accelerated AWS Growth:** Amazon Web Services (AWS) revenue accelerated to 20.2% year-over-year growth in the third quarter of 2025, up from 17.5% in the second quarter. AWS also has a reported $200 billion backlog. Analysts project AWS revenue to reach $128.1 billion in 2025, growing to $348.5 billion in 2030.
*   **AI Demand:** Amazon is well-positioned amid surging AI demand and cloud infrastructure growth. The company's recent multi-year deal with OpenAI, involving a $38 billion commitment for OpenAI to leverage AWS compute, further highlights its role in the AI space. AWS's Trainium chips business is also experiencing rapid growth.
*   **E-commerce and Advertising:** Amazon's retail operations continue to be a primary revenue source, and its advertising business is expected to continue outperforming expectations.
*   **Warehouse Automation:** Robotics deployment across fulfillment centers is expected to deliver substantial cost savings and improve fulfillment costs by 20% to 40%.

While the overall outlook is strongly bullish, some potential bearish considerations exist. Concerns about the company's enormous AI capital expenditures remain. Additionally, while AWS growth is strong, it may not be as fast as competing cloud services like Microsoft Azure and Google Cloud, posing a risk of market share losses before 2030 if Amazon cannot maintain its competitive edge. Some short-term technical indicators have also shown a "general sell signal" from the relation between long-term and short-term moving averages, where the long-term average is above the short-term average. However, this is largely overshadowed by other positive signals.

In [8]:
response = chat.send_message('''What is mgm studio's stock ticker symbol?''')
Markdown(response.text)

MGM Studios, as an independent entity, does not have a publicly traded stock ticker symbol. MGM Holdings, the parent company of MGM Studios, was acquired by Amazon in 2022. Therefore, MGM Studios is now part of Amazon, and its financial performance is reflected within Amazon's overall operations (NASDAQ: AMZN).

However, there is a publicly traded company called **MGM Resorts International**, which operates casinos and resorts and uses the stock ticker symbol **MGM** on the New York Stock Exchange (NYSE). This company is distinct from the film and television studio.

In [9]:
response = chat.send_message('''What was the last open,close,high,low data for AMZN again?''')
Markdown(response.text)

As of Friday, November 28, 2025, the last recorded trading day, Amazon (AMZN) stock had the following data:

*   **Close/Last Price:** $233.22
*   **Open Price:** $231.24
*   **High Price:** $233.28
*   **Low Price:** $230.22

In [10]:
response = chat.send_message('''What is AMZN open,close,high,low data for the past month? 
Present the data with multiple columns for display in markdown.''')
Markdown(response.text)

Here is the open, close, high, and low data for Amazon (AMZN) stock for the past month, from October 31, 2025, to November 28, 2025:

| Date         | Open (USD) | Close (USD) | High (USD) | Low (USD) |
| :----------- | :--------- | :---------- | :--------- | :-------- |
| Nov 28, 2025 | $231.24    | $233.22     | $233.28    | $230.22   |
| Nov 26, 2025 | $230.74    | $229.16     | $231.75    | $228.77   |
| Nov 25, 2025 | $226.38    | $229.67     | $230.52    | $223.80   |
| Nov 24, 2025 | $222.56    | $226.28     | $227.33    | $222.27   |
| Nov 21, 2025 | $216.34    | $220.69     | $222.21    | $215.18   |
| Nov 20, 2025 | $227.05    | $217.14     | $227.41    | $216.74   |
| Nov 19, 2025 | $223.74    | $222.69     | $223.74    | $218.52   |
| Nov 18, 2025 | $228.10    | $222.55     | $230.20    | $222.42   |
| Nov 17, 2025 | $233.25    | $232.87     | $234.60    | $229.19   |
| Nov 14, 2025 | $235.06    | $234.69     | $238.73    | $232.89   |
| Nov 13, 2025 | $243.05    | $237.58     | $243.75    | $236.50   |
| Nov 12, 2025 | $250.24    | $244.20     | $250.37    | $243.75   |
| Nov 11, 2025 | $248.41    | $249.10     | $249.75    | $247.23   |
| Nov 10, 2025 | $248.34    | $248.40     | $251.75    | $245.59   |
| Nov 07, 2025 | $242.90    | $244.41     | $244.90    | $238.49   |
| Nov 06, 2025 | $249.16    | $243.04     | $250.38    | $242.17   |
| Nov 05, 2025 | $249.03    | $250.20     | $251.00    | $246.16   |
| Nov 04, 2025 | $250.38    | $249.32     | $257.01    | $248.66   |
| Nov 03, 2025 | $255.36    | $254.00     | $258.60    | $252.90   |
| Oct 31, 2025 | $250.10    | $244.22     | $250.50    | $243.98   |
| Oct 30, 2025 | $227.06    | $222.86     | $228.44    | $222.75   |
| Oct 29, 2025 | $231.67    | $230.30     | $231.67    | $228.22   |
| Oct 28, 2025 | $228.22    | $229.25     | $231.49    | $226.21   |
| Oct 27, 2025 | $227.66    | $226.97     | $228.40    | $225.54   |
| Oct 26, 2025 | $221.09    | $224.21     | $224.21    | $221.09   |

Please note that stock market data can fluctuate rapidly, and the prices provided are based on the last available information from the specified dates.

## Previously on Kaggle: StockChat 1.0

### Validation BaseModels

In [11]:
# Validation BaseModels in pydantic schema.
class RestStatus(Enum):
    OK = "OK"
    DELAY = "DELAYED"
    NONE = "NOT_FOUND"
    AUTH = "NOT_AUTHORIZED"

class StopGeneration(BaseModel):
    result: str = Api.Const.Stop()

class RestResultPoly(BaseModel):
    request_id: Optional[str] = None
    count: Optional[int] = None
    next_url: Optional[str] = None
    status: RestStatus  

class MarketSession(Enum):
    PRE = "pre-market"
    REG = "regular"
    POST = "post-market"
    CLOSED = "closed"
    NA = "not applicable"

class MarketEvent(Enum):
    PRE_OPEN = 0
    REG_OPEN = 1
    REG_CLOSE = 2
    POST_CLOSE = 3
    LAST_CLOSE = 4

class AssetClass(Enum):
    STOCKS = "stocks"
    OPTION = "options"
    CRYPTO = "crypto"
    FOREX = "fx"
    INDEX = "indices"
    OTC = "otc"

class SymbolType(Enum):
    COMMON = "Common Stock"
    ETP = "ETP"
    ADR = "ADR"
    REIT = "REIT"
    DELISTED = ""
    CEF = "Closed-End Fund"
    UNIT = "Unit"
    RIGHT = "Right"
    EQUITY = "Equity WRT"
    GDR = "GDR"
    PREF = "Preference"
    CDI = "CDI"
    NVDR = "NVDR"
    REG = "NY Reg Shrs"
    MLP = "MLP"
    MUTUAL = "Mutual Fund"

class Locale(Enum):
    US = "us"
    GLOBAL = "global"

class Sentiment(Enum):
    V_POS = "very positive"
    POSITIVE = "positive"
    NEUTRAL_P = "neutral/positive"
    NEUTRAL_SP = "neutral/slightly positive"
    NEUTRAL = "neutral"
    NEUTRAL_SN = "neutral/slightly negative"
    NEUTRAL_N = "neutral/negative"
    MIXED = "mixed"
    NEGATIVE = "negative"
    V_NEG = "very negative"

class Trend(Enum):
    S_BUY = "strong-buy"
    BUY = "buy"
    HOLD = "hold"
    SELL = "sell"
    S_SELL = "strong-sell"

class MarketCondition(Enum):
    BULL = "bullish"
    BULLN = "cautiously bullish"
    HOLD = "hold"
    BEARN = "cautiously bearish"
    BEAR = "bearish"

class GeneratedEvent(BaseModel):
    last_close: str
    pre_open: str
    reg_open: str
    reg_close: str
    post_close: str
    timestamp: Optional[str] = None
    is_holiday: Optional[bool] = None

    def model_post_init(self, *args, **kwargs) -> None:
        if self.timestamp is None:
            self.timestamp = datetime.now(self.tz()).strftime('%c')
        if self.is_holiday is None:
            self.is_holiday = False

    def session(self, with_date: Optional[str] = None) -> MarketSession:
        if with_date is None:
            with_date = datetime.now(self.tz()).strftime('%c')
        compare = parse(with_date)
        if self.is_holiday or compare.weekday() > 4: # weekend
            return MarketSession.CLOSED
        events = [parse(event).time() for event in [self.pre_open,self.reg_open,self.reg_close,self.post_close]]
        if compare.time() < events[0]:
            return MarketSession.CLOSED
        else:
            session = MarketSession.NA
            if compare.time() >= events[0]:
                session = MarketSession.PRE
            if compare.time() >= events[1]:
                session = MarketSession.REG
            if compare.time() >= events[2]:
                session = MarketSession.POST
            if compare.time() >= events[3]:
                session = MarketSession.CLOSED
        return session

    def is_open(self) -> bool:
        return self.session() != MarketSession.CLOSED

    def has_update(self) -> bool:
        datetime_now = datetime.now(self.tz())
        self_ts = parse(self.timestamp)
        # Re-generate events for a new day.
        if datetime_now.day > self_ts.day:
            return True
        # No updates on holidays or when generated after post_close.
        if self.is_holiday or self_ts.time() >= parse(self.post_close).time():
            return False
        # Compare current time to generated event times.
        for event in [self.pre_open,self.reg_open,self.reg_close]:
            if datetime_now.time() > parse(event).time():
                return True
        # Current time is before pre_open.
        return False

    @classmethod
    def tz(cls):
        return pytz.timezone('US/Eastern') # Exchanges data is in eastern time.
    
    @classmethod
    def apply_fix(cls, value, fix: datetime) -> tuple[str, datetime]:
        api.validation_fail()
        value = fix.strftime('%c')
        return value, fix
    
    @field_validator("last_close")
    def valid_close(cls, value):
        date_gen = parse(value) # Generated close is in eastern time and tzinfo naive.
        date_now = parse(datetime.now(cls.tz()).strftime('%c')) # Need now in same format as generated.
        # Soft-pass: when actual session is closed after post-market
        if date_now.day == date_gen.day+1 and date_now.weekday() <= 4:
            date_fix = date_gen.replace(day=date_now.day)
            if date_fix.timestamp() < date_now.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use today's close
        # Soft-pass: when actual session is open post-market
        if date_now.day == date_gen.day and date_now.timestamp() < date_gen.timestamp():
            if date_now.weekday() > 0:
                date_fix = date_gen.replace(day=date_now.day-1)
            else:
                date_fix = date_gen.replace(day=date_now.day-3)
            if date_now.timestamp() > date_fix.timestamp():
                value, date_gen = cls.apply_fix(value, date_fix) # soft-pass: use previous close
        if date_now.weekday() == 0 or date_now.weekday() == 1 and date_gen.weekday() <= 4: # 0=monday, 4=friday
            return value # pass: generated thurs/friday on a monday/tues
        elif date_now.weekday() > 0 and date_now.weekday() <= 4 and date_gen.weekday() <= date_now.weekday()-1:
            return value # pass: generated yesterday/prior on a tues-fri
        elif date_now.weekday() > 4 and date_gen.weekday() <= 4:
            return value # pass: generated thurs/friday on a weekend
        elif date_now.day == date_gen.day and date_now.timestamp() > date_gen.timestamp():
            return value # pass: generated today after closed
        elif date_now.timestamp() < date_gen.timestamp():
            raise ValueError("last close cannot be a future value")
        else:
            raise ValueError("generated invalid last close")
        api.validation_fail()

class VectorStoreResult(BaseModel):
    docs: str
    dist: Optional[float] # requires query
    meta: Optional[dict]  # requires get or query
    store_id: str

class Aggregate(RestResultPoly):
    symbol: str
    open: float
    high: float
    low: float
    close: float
    volume: int
    otc: Optional[bool] = None
    preMarket: Optional[float] = None
    afterHours: Optional[float] = None

class DailyCandle(Aggregate):
    from_date: str

class AggregateWindow(BaseModel):
    o: float
    h: float
    l: float
    c: float
    v: int # traded volume
    n: Optional[int] = None # transaction count
    vw: Optional[float] = None # volume weighted average price
    otc: Optional[bool] = None
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        if len(str(value)) == 13:
            return int(value/1000)
        return value

class CustomCandle(RestResultPoly): 
    ticker: str
    adjusted: bool
    queryCount: int
    resultsCount: int
    results: list[AggregateWindow]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[AggregateWindow]:
        return self.results
    
class MarketStatus(BaseModel):
    exchange: str
    holiday: Optional[str] = None
    isOpen: bool
    session: Optional[MarketSession] = None
    t: int
    timezone: str

    def model_post_init(self, *args, **kwargs) -> None:
        if self.session is None:
            self.session = MarketSession.CLOSED
        if self.holiday is None:
            self.holiday = MarketSession.NA.value

class MarketStatusResult(BaseModel):
    results: MarketStatus

    def get(self) -> MarketStatus:
        return self.results

class Symbol(BaseModel):
    description: str
    displaySymbol: str
    symbol: str
    type: SymbolType

class SymbolResult(BaseModel):
    count: int
    result: list[Symbol]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.result)

    def get(self) -> list[Symbol]:
        return self.result

class Quote(BaseModel):
    c: float
    d: float
    dp: float
    h: float
    l: float
    o: float
    pc: float
    t: int

    @field_validator("t")
    def valid_t(cls, value):
        if not value > 0:
            raise ValueError("invalid timestamp")
        return value

class PeersResult(BaseModel):
    results: list[str]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[str]:
        return self.results

class BasicFinancials(BaseModel):
    metric: dict
    metricType: str
    series: dict
    symbol: str

class Insight(BaseModel):
    sentiment: Sentiment|MarketCondition
    sentiment_reasoning: str
    ticker: str

class Publisher(BaseModel):
    favicon_url: Optional[str]
    homepage_url: str
    logo_url: str
    name: str

class NewsSummary(BaseModel):
    title: str
    summary: Optional[str]
    insights: Optional[list[Insight]]
    published_utc: str

class NewsTypePoly(BaseModel):
    amp_url: Optional[str] = None
    article_url: str
    title: str
    author: str
    description: Optional[str] = None
    id: str
    image_url: Optional[str] = None
    insights: Optional[list[Insight]] = None
    keywords: Optional[list[str]] = None
    published_utc: str
    publisher: Publisher
    tickers: list[str]

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.description,
                           insights=self.insights,
                           published_utc=self.published_utc)

class NewsResultPoly(RestResultPoly):
    results: list[NewsTypePoly]

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypePoly]:
        return self.results

class NewsTypeFinn(BaseModel):
    category: str
    datetime: int
    headline: str
    id: int
    image: str
    related: str # symbol
    source: str
    summary: str
    url: str

    def summary(self):
        return NewsSummary(title=self.headline,
                           summary=self.summary,
                           insights=None,
                           published_utc=self.datetime)

class NewsResultFinn(BaseModel):
    results: list[NewsTypeFinn]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[NewsTypeFinn]:
        return self.results

class NewsTypeGenerated(BaseModel):
    title: str
    summary: str
    insights: list[Insight]
    keywords: list[str]
    source: Publisher
    published_utc: str
    tickers: list[str]
    url: str

    def summary(self):
        return NewsSummary(title=self.title,
                           summary=self.summary,
                           insights=self.insights,
                           published_utc=self.published_utc)

class TickerOverview(BaseModel):
    ticker: str
    name: str
    market: AssetClass
    locale: Locale
    primary_exchange: Optional[str] = None
    active: bool
    currency_name: str
    cik: Optional[str] = None
    composite_figi: Optional[str] = None
    share_class_figi: Optional[str] = None
    market_cap: Optional[int|float] = None
    phone_number: Optional[str] = None
    address: Optional[dict] = None
    description: Optional[str] = None
    sic_code: Optional[str] = None
    sic_description: Optional[str] = None
    ticker_root: Optional[str] = None
    homepage_url: Optional[str] = None
    total_employees: Optional[int] = None
    list_date: Optional[str] = None
    branding: Optional[dict] = None
    share_class_shares_outstanding: Optional[int] = None
    weighted_shares_outstanding: Optional[int] = None
    round_lot: Optional[int] = None

class OverviewResult(RestResultPoly):
    results: TickerOverview

    def get(self) -> TickerOverview:
        return self.results

class RecommendationTrend(BaseModel):
    buy: int
    hold: int
    period: str
    sell: int
    strongBuy: int
    strongSell: int
    symbol: str

class TrendsResult(BaseModel):
    results: list[RecommendationTrend]
    count: Optional[int] = None

    def model_post_init(self, *args, **kwargs) -> None:
        self.count = len(self.results)

    def get(self) -> list[RecommendationTrend]:
        return self.results

### Contents Memory

In [12]:
# A contents-memory object.
class Memory:
    def __init__(self):
        self.system = f"""Give a concise, and detailed summary. Use information that you learn from the API responses.
        Use your tools and function calls according to the rules. Convert any all-upper case identifiers
        to proper case in your response. Convert any abbreviated or shortened identifiers to their full forms.
        Convert timestamps according to the rules before including them. Think step by step.
        """
        self.revery = {}
        self.contents = []
        self.prompt = None
        self.summary = None
        self.response = None
    
    def set_prompt(self, prompt):
        self.prompt = f"""
        The current date and time is: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
        
        {prompt}
        """
        self.contents = [types.Content(role="user", parts=[types.Part(text=self.prompt)])]

    def set_reason(self, step):
        # Append the model's reasoning part.
        self.contents.append(types.Content(role="model", parts=[types.Part(thought=True,text=step)]))

    def append_code(self, prompt, code_response_parts):
        subroutine_content = [types.Content(role="user", parts=[types.Part(text=prompt)]),
                              types.Content(role="model", parts=code_response_parts)]
        # Append the model's generated code and execution result.
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = { 
            "contents": subroutine_content
        }

    def update_contents(self, function_call, api_response_part):
        # Append the model's function call part.
        self.contents.append(types.Content(role="model", parts=[types.Part(function_call=function_call)])) 
        # Append the api response part.
        self.contents.append(types.Content(role="user", parts=[api_response_part]))

    def set_summary(self, summary):
        self.summary = summary
        self.contents.append(types.Content(role="model", parts=[types.Part(text=summary)]))
        self.revery[datetime.now(GeneratedEvent.tz()).strftime('%c')] = {
            "prompt": self.prompt, 
            "summary": self.summary, 
            "contents": self.contents
        }
        self.contents = []

memory = Memory()

### Retrieval-Augmented Generation

In [13]:
# Define tool: retrieval-augmented generation.
# - using Chroma and text-embedding-004 for storage and retrieval
# - using gemini-2.0-flash for augmented generation
class RetrievalAugmentedGenerator:
    chroma_client = chromadb.PersistentClient(path="vector_db")
    config_temp = types.GenerateContentConfig(temperature=0.0)
    exchange_codes: Optional[dict] = None
    exchange_lists: dict = {}
    events: dict = {}
    holidays: dict = {}

    def __init__(self, genai_client, collection_name):
        self.client = genai_client
        self.embed_fn = GeminiEmbedFunction(genai_client)
        self.db = self.chroma_client.get_or_create_collection(
            name=collection_name, 
            embedding_function=self.embed_fn,  # type: ignore
            metadata={"hnsw:space": "cosine"})
        logging.getLogger("chromadb").setLevel(logging.ERROR) # suppress warning on existing id
        self.set_holidays("US", ["09-01-2025","10-13-2025","11-11-2025","11-27-2025","12-25-2025"])
        #self.generated_events("US")

    def set_holidays(self, exchange_code: str, holidays: list):
        self.holidays[exchange_code] = [datetime.strptime(h, "%m-%d-%Y").date() for h in holidays]

    def get_exchange_codes(self, with_query: Optional[str] = None):
        gen = None
        if with_query and with_query not in self.exchange_lists.keys():
            gen = tqdm(total=1, desc="Generate exchange codes with_query")
            data = self.get_exchanges_csv(
                f"""What is the {with_query} exchange code? Return only the exchange codes 
                as a list in string form. Just the list string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_lists[with_query] = ast.literal_eval(data.parts[-1].text)
        elif with_query is None and self.exchange_codes is None:
            gen = tqdm(total=1, desc="Generate exchange codes")
            data = self.get_exchanges_csv(
                """Give me a dictionary in string form. It must contain key:value pairs 
                mapping exchange code to name. Just the dictionary string. 
                Omit all other information or details. Do not chat or use sentences.""").candidates[0].content
            self.exchange_codes = ast.literal_eval(data.parts[-1].text.strip("\\`"))
        if gen:
            gen.update(1)
        return self.exchange_lists[with_query] if with_query else self.exchange_codes

    def get_event_date(self, event_t: str, exchange_code: str, event: MarketEvent):
        current_dt_str = datetime.now(GeneratedEvent.tz()).strftime('%c')
        current_dt = datetime.strptime(current_dt_str, "%a %b %d %H:%M:%S %Y")
        current_t_str = datetime.now(GeneratedEvent.tz()).strftime('%H:%M:%S')
        current_t = datetime.strptime(current_t_str, "%H:%M:%S").time()
        event_time = parse(event_t).time()
        gen_datetime = None
        if event is MarketEvent.LAST_CLOSE:
            last_close_day = current_dt.date() - timedelta(days=0 if current_t > event_time else 1)
            # Loop backwards to find the last valid trading day (not a weekend or holiday).
            while last_close_day.weekday() >= 5 or last_close_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                last_close_day -= timedelta(days=1)
            # Combine the date and time.
            gen_datetime = datetime.combine(last_close_day, event_time)
        else:
            next_event_day = current_dt.date() + timedelta(days=0 if current_t < event_time else 1)
            # Loop forward to find the next valid trading day (not a weekend or holiday).
            while next_event_day.weekday() >= 5 or next_event_day in self.holidays[exchange_code]: # 5 = Sat, 6 = Sun
                next_event_day += timedelta(days=1)
            # Combine date and time.
            gen_datetime = datetime.combine(next_event_day, event_time)
        # Format the result as requested.
        return gen_datetime.strftime('%a %b %d %X %Y')

    def generate_event(self, exchange_code: str, event: MarketEvent):
        if event is MarketEvent.LAST_CLOSE or event is MarketEvent.POST_CLOSE:
            prompt = f"""What is the closing time including post_market hours."""
        elif event is MarketEvent.PRE_OPEN or event is MarketEvent.REG_OPEN:
            is_pre = "including" if event is MarketEvent.PRE_OPEN else "excluding"
            prompt = f"""What is the opening time {is_pre} pre_market hours."""
        elif event is MarketEvent.REG_CLOSE:
            prompt = f"""What is the closing time excluding post_market hours."""
        prompt = f"""Answer based on your knowledge of exchange operating hours.
            Do not answer in full sentences. Omit all chat and provide the answer only.
            The fields pre_market and post_market both represent extended operating hours.

            The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}
            
            Consider the {exchange_code} exchange's operating hours.
            {prompt}
            
            Answer with the time in this format: '%H:%M:%S'.
            Omit all other chat and details. Do not use sentences."""
        progress = tqdm(total=1, desc=f"Generate {exchange_code}->{event}")
        response = self.get_exchanges_csv(prompt).candidates[0].content
        try:
            if Api.Const.Stop() in f"{response.parts[-1].text}":
                self.generate_event_failed(progress, exchange_code, event)
            else:
                response = self.get_event_date(response.parts[-1].text, exchange_code, event)
                progress.update(1)
                return response
        except Exception as e:
            self.generate_event_failed(progress, exchange_code, event)

    def generate_event_failed(self, progress: tqdm, exchange_code: str, event: MarketEvent):
        progress.close()
        api.generation_fail()
        time.sleep(api.dt_between)
        return self.generate_event(exchange_code, event)

    def generated_events(self, exchange_code: str) -> GeneratedEvent:
        # Check for an existing GeneratedEvent object having updates.
        if exchange_code in self.events.keys() and self.events[exchange_code].has_update():
            event_obj = self.events[exchange_code]
            event_state = [(event_obj.pre_open, MarketEvent.PRE_OPEN),
                           (event_obj.reg_open, MarketEvent.REG_OPEN),
                           (event_obj.reg_close, MarketEvent.REG_CLOSE),
                           (event_obj.post_close, MarketEvent.POST_CLOSE)]
            # Need now in same format as generated.
            datetime_now = parse(datetime.now(event_obj.tz()).strftime('%c'))
            gen_ts = parse(event_obj.timestamp)
            # Re-generate events when day changes.
            if datetime_now.day > gen_ts.day:
                del self.events[exchange_code]
                return self.generated_events(exchange_code)
            # Update changed events on trading days.
            for e in event_state:
                if datetime_now > parse(e[0]):
                    event_dt = self.generate_event(exchange_code, e[1])
                    match e[1]:
                        case MarketEvent.PRE_OPEN:
                            event_obj.pre_open = event_dt
                        case MarketEvent.REG_OPEN:
                            event_obj.reg_open = event_dt
                        case MarketEvent.REG_CLOSE:
                            event_obj.reg_close = event_dt
                        case MarketEvent.POST_CLOSE:
                            event_obj.post_close = event_dt
            event_obj.timestamp = datetime.now(event_obj.tz()).strftime('%c')
            self.events[exchange_code] = event_obj
        # Generate events for an exchange code not in cache.
        elif exchange_code not in self.events.keys():
            self.events[exchange_code] = GeneratedEvent(
                last_close=self.generate_event(exchange_code, MarketEvent.LAST_CLOSE),
                pre_open=self.generate_event(exchange_code, MarketEvent.PRE_OPEN),
                reg_open=self.generate_event(exchange_code, MarketEvent.REG_OPEN),
                reg_close=self.generate_event(exchange_code, MarketEvent.REG_CLOSE),
                post_close=self.generate_event(exchange_code, MarketEvent.POST_CLOSE),
                is_holiday=datetime.now().date() in self.holidays[exchange_code])
        return self.events[exchange_code]

    def set_holiday_event(self, exchange_code: str):
        self.generated_events(exchange_code).is_holiday = True

    def last_market_close(self, exchange_code: str):
        return self.generated_events(exchange_code).last_close

    def add_documents_list(self, docs: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        metas=[{"source": doc.metadata["source"]} for doc in docs]
        content=[doc.page_content for doc in docs]
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate document embedding")

    def add_api_document(self, query: str, api_response: str, topic: str, source: str = "add_api_document"):
        self.embed_fn.document_mode = True # Switch to document mode.
        splitter = RecursiveJsonSplitter(max_chunk_size=Api.Const.ChunkMax())
        docs = splitter.create_documents(texts=[api_response], convert_lists=True)
        ids = list(map(str, range(self.db.count(), self.db.count()+len(docs))))
        content = [json.dumps(doc.page_content) for doc in docs]
        metas = [{"source": source, "topic": topic}]*len(docs)
        tqdm(self.db.add(ids=ids, documents=content, metadatas=metas), desc="Generate api embedding")

    def add_peers_document(self, query: str, names: list, topic: str, source: str, group: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        peers = {"symbol": topic, "peers": names}
        tqdm(self.db.add(ids=str(self.db.count()),
                         documents=[json.dumps(peers)],
                         metadatas=[{"source": source, "topic": topic, "group": group}]),
             desc="Generate peers embedding")

    def get_peers_document(self, query: str, topic: str, group: str):
        return self.get_documents_list(query, where={"$and": [{"group": group}, {"topic": topic}]})

    def add_rest_chunks(self, chunks: list, topic: str, source: str, ids: Optional[list[str]] = None,
                        meta_opt: Optional[list[dict]] = None, is_update: bool = True):
        self.embed_fn.document_mode = True # Switch to document mode
        if ids is None:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(chunks))))
        if isinstance(chunks[0], BaseModel):
            docs = [model.model_dump_json() for model in chunks]
        else:
            docs = [json.dumps(obj) for obj in chunks]
        meta_base = {"source": source, "topic": topic}
        if meta_opt is not None:
            for m in meta_opt:
                m.update(meta_base)
        metas = [meta_base]*len(chunks) if meta_opt is None else meta_opt
        if is_update:
            tqdm(self.db.upsert(ids=ids, documents=docs, metadatas=metas), desc="Upsert chunks embedding")
        else:
            tqdm(self.db.add(ids=ids, documents=docs, metadatas=metas), desc="Add chunks embedding")

    def get_market_status(self, exchange_code: str) -> tuple[list[VectorStoreResult], bool]: # result, has rest update
        self.embed_fn.document_mode = False # Switch to query mode.
        stored = self.stored_result(self.db.get(where={
            "$and": [{"exchange": exchange_code}, {"topic": "market_status"}]}))
        if len(stored) == 0:
            return stored, True
        # Check for a daily market status update.
        status = json.loads(stored[0].docs)
        gen_day = parse(self.generated_events(exchange_code).timestamp).day
        store_day = parse(stored[0].meta['timestamp']).day
        if status["holiday"] != MarketSession.NA.value and gen_day == store_day:
            return stored, False
        elif gen_day > store_day:
            return stored, True
        # Update with generated events to avoid rest api requests.
        status["session"] = self.generated_events(exchange_code).session().value
        status["isOpen"] = self.generated_events(exchange_code).is_open()
        stored[0].docs = json.dumps(status)
        return stored, False

    def get_basic_financials(self, query: str, topic: str, source: str = "get_financials_1"):
        return self.get_documents_list(
            query, max_sources=200, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_quote_document(self, query: str, quote: str, topic: str, timestamp: int, source: str):
        self.embed_fn.document_mode = True # Switch to document mode.
        tqdm(self.db.add(ids=str(self.db.count()), 
                             documents=[quote], 
                             metadatas=[{"source": source, "topic": topic, "timestamp": timestamp}]), 
             desc="Generate quote embedding")

    def get_api_documents(self, query: str, topic: str, source: str = "add_api_document", 
                          meta_opt: Optional[list[dict]] = None):
        where = [{"source": source}, {"topic": topic}]
        if meta_opt is None:
            return self.get_documents_list(query, where={"$and": where})
        else:
            for meta in meta_opt:
                for k,v in meta.items():
                    where.append({k: v})
            return self.get_documents_list(query, where={"$and": where})

    def query_api_documents(self, query: str, topic: str, source: str = "add_api_document"):
        return self.generate_answer(query, where={"$and": [{"source": source}, {"topic": topic}]})

    def add_grounded_document(self, query: str, topic: str, result):
        self.embed_fn.document_mode = True # Switch to document mode.
        chunks = result.candidates[0].grounding_metadata.grounding_chunks
        supports = result.candidates[0].grounding_metadata.grounding_supports
        if supports is not None: # Only add grounded documents which have supports
            grounded_text = [f"{s.segment.text}" for s in supports]
            source = [f"{c.web.title}" for c in chunks]
            score = [f"{s.confidence_scores}" for s in supports]
            tqdm(self.db.add(ids=str(self.db.count()),
                             documents=json.dumps(grounded_text),
                             metadatas=[{"source": ", ".join(source),
                                         "confidence_score": ", ".join(score),
                                         "topic": topic,
                                         "question": query}]),
                 desc="Generate grounding embedding")

    def get_grounding_documents(self, query: str, topic: str):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(self.db.get(where={"$and": [{"question": query}, {"topic": topic}]}))
            
    def add_wiki_documents(self, title: str, wiki_chunks: list):
        self.embed_fn.document_mode = True # Switch to document mode.
        result = self.get_wiki_documents(title)
        if len(result) == 0:
            ids = list(map(str, range(self.db.count(), self.db.count()+len(wiki_chunks))))
            metas=[{"title": title, "source": "add_wiki_documents"}]*len(wiki_chunks)
            tqdm(self.db.add(ids=ids, documents=wiki_chunks, metadatas=metas), desc="Generate wiki embeddings")

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_with_wiki_passages(self, query: str, title: str, passages: list[str]):
        return self.generate_answer(query, where={"title": title}, passages=passages)
    
    def get_wiki_documents(self, title: Optional[str] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        if title is None:
            return self.stored_result(self.db.get(where={"source": "add_wiki_document"}))
        else:
            return self.stored_result(self.db.get(where={"title": title}))

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_documents_list(self, query: str, max_sources: int = 5000, where: Optional[dict] = None):
        self.embed_fn.document_mode = False # Switch to query mode.
        return self.stored_result(
            self.db.query(query_texts=[query], 
                          n_results=max_sources, 
                          where=where), 
            is_query = True)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_exchanges_csv(self, query: str):
        return self.generate_answer(query, max_sources=100, where={"source": "exchanges.csv"})

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def generate_answer(self, query: str, max_sources: int = 10, 
                        where: Optional[dict] = None, passages: Optional[list[str]] = None):
        stored = self.get_documents_list(query, max_sources, where)
        query_oneline = query.replace("\n", " ")
        prompt = f"""You're an expert writer. You understand how to interpret html and markdown. You will accept the
        question below and answer based only on the passages. Never mention the passages in your answers. Be sure to 
        respond in concise sentences. Include all relevant background information when possible. If a passage is not 
        relevant to the answer you must ignore it. If no passage answers the question respond with: I don't know.

        QUESTION: {query_oneline}
        
        """
        # Add the retrieved documents to the prompt.
        stored_docs = [passage.docs for passage in stored]
        for passage in stored_docs if passages is None else stored_docs + passages:
            passage_oneline = passage.replace("\n", " ")
            prompt += f"PASSAGE: {passage_oneline}\n"
        # Generate the response.
        response = api.retriable(
            self.client.models.generate_content,
            model=api(Api.Model.GEN),
            config=self.config_temp,
            contents=prompt)
        # Check for generated code and store in memory.
        content = response.candidates[0].content
        if len(content.parts) > 1 and content.parts[0].executable_code:
            memory.append_code(prompt, content.parts)
        return response

    def stored_result(self, result, is_query: bool = False) -> list[VectorStoreResult]:
        try:
            results = []
            if len(result["documents"]) == 0:
                return results
            if isinstance(result["documents"][0], list):
                for i in range(len(result["documents"][0])):
                    obj = VectorStoreResult(docs=result["documents"][0][i],
                                            dist=result["distances"][0][i] if is_query else None,
                                            meta=result["metadatas"][0][i],
                                            store_id=result["ids"][0][i])
                    results.append(obj)
            else:
                results.append(
                    VectorStoreResult(docs=result["documents"][0],
                                      dist=result["distances"][0] if is_query else None,
                                      meta=result["metadatas"][0],
                                      store_id=result["ids"][0]))
            return results
        except Exception as e:
            raise e

### Wiki Grounding

In [14]:
# Define tool: wiki-grounding generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by similarity to topic
# - retrieve existing groundings by similarity to topic
class WikiGroundingGenerator:   
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl
        with warnings.catch_warnings():
            warnings.simplefilter("ignore") # suppress beta-warning
            self.splitter = HTMLSemanticPreservingSplitter(
                headers_to_split_on=[("h2", "Main Topic"), ("h3", "Sub Topic")],
                separators=["\n\n", "\n", ". ", "! ", "? "],
                max_chunk_size=Api.Const.ChunkMax(),
                chunk_overlap=50,
                preserve_links=True,
                preserve_images=True,
                preserve_videos=True,
                preserve_audio=True,
                elements_to_preserve=["table", "ul", "ol", "code"],
                denylist_tags=["script", "style", "head"],
                custom_handlers={"code": self.code_handler},
            )

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_wiki_documents(topic)
        if len(stored) > 0:
            return self.rag.generate_with_wiki_passages(query, topic, [chunk.docs for chunk in stored]).text
        else:
            pages = wikipedia.search(topic + " company")
            if len(pages) > 0:
                p_topic_match = 0.80
                for i in range(len(pages)):
                    if tqdm(api.similarity([topic + " company", pages[i]]) > p_topic_match, 
                            desc= "Score wiki search by similarity to topic"):
                        page_html = Api.get(f"https://en.wikipedia.org/wiki/{pages[i]}")
                        chunks = [chunk.page_content for chunk in self.splitter.split_text(page_html)]
                        self.rag.add_wiki_documents(topic, chunks)
                        return self.rag.generate_with_wiki_passages(query, topic, chunks).text
            return Api.Const.Stop()

    def code_handler(self, element: Tag) -> str:
        data_lang = element.get("data-lang")
        code_format = f"<code:{data_lang}>{element.get_text()}</code>"
        return code_format

### Search Grounding

In [15]:
# Define tool: search-grounding generation.
# - using gemini-2.0-flash with GoogleSearch tool for response generation
# - using a RAG-implementation to store groundings
# - create new groundings by exact match to topic
# - retrieve existing groundings by similarity to topic
class SearchGroundingGenerator:
    config_ground = types.GenerateContentConfig(
        tools=[types.Tool(google_search=types.GoogleSearch())],
        temperature=0.0
    )
    
    def __init__(self, genai_client, rag_impl):
        self.client = genai_client
        self.rag = rag_impl

    def generate_answer(self, query: str, topic: str):
        stored = self.rag.get_grounding_documents(query, topic)
        if len(stored) > 0:
            for i in range(len(stored)):
                meta_q = stored[i].meta["question"]
                p_ground_match = 0.95 # This can be really high ~ 95-97%
                if tqdm(api.similarity([query, meta_q]) > p_ground_match,
                        desc="Score similarity to stored grounding"):
                    return ast.literal_eval(stored[i].docs)
        return self.get_grounding(query, topic)

    @retry.Retry(
        predicate=is_retriable,
        initial=2.0,
        maximum=64.0,
        multiplier=2.0,
        timeout=600,
    )
    def get_grounding(self, query: str, topic: str):
        contents = [types.Content(role="user", parts=[types.Part(text=query)])]
        contents += f"""
        You're a search assistant that provides answers to questions about {topic}.
        Do not discuss alternative topics of interest. Do not discuss similar topics.
        You will provide answers that discuss only {topic}. 
        You may discuss the owner or parent of {topic} when no other answer is possible.
        Otherwise respond with: I don't know."""
        response = api.retriable(self.client.models.generate_content, 
                                 model=api(Api.Model.GEN), 
                                 config=self.config_ground, 
                                 contents=contents)
        if response.candidates[0].grounding_metadata.grounding_supports is not None:
            if self.is_consistent(query, topic, response.text):
                self.rag.add_grounded_document(query, topic, response)
                return response.text 
        return Api.Const.Stop() # Empty grounding supports or not consistent in response

    def is_consistent(self, query: str, topic: str, model_response: str) -> bool:
        topic = topic.replace("'", "")
        id_strs = topic.split()
        if len(id_strs) == 1:
            matches = re.findall(rf"{id_strs[0]}[\s,.]+\S+", query)
            if len(matches) > 0:
                topic = matches[0]
        compound_match = re.findall(rf"{id_strs[0]}[\s,.]+\S+", model_response)
        model_response = model_response.replace("'", "")
        if len(compound_match) == 0 and topic in model_response:
            return True # not a compound topic id and exact topic match
        for match in compound_match:
            if topic not in match:
                return False
        return True # all prefix matches contained topic

### Rest Grounding

In [16]:
# Rest api-helpers to manage request-per-minute limits.
# - define an entry for each endpoint limit
# - init rest tool with limits to create blocking queues
# - apply a limit to requests with rest_tool.try_url
class ApiLimit(Enum):
    FINN = "finnhub.io",50
    POLY = "polygon.io",4 # (id_url,rpm)

class BlockingUrlQueue:
    on_cooldown = False
    cooldown = None
    cooldown_start = None
    
    def __init__(self, rest_fn: Callable, per_minute: int):
        self.per_minute_max = per_minute
        self.quota = per_minute
        self.rest_fn = rest_fn

    def push(self, rest_url: str):
        if not self.on_cooldown:
            self.cooldown = Timer(60, self.reset_quota)
            self.cooldown.start()
            self.cooldown_start = time.time()
            self.on_cooldown = True
        if self.quota > 0:
            self.quota -= 1
            time.sleep(0.034) # ~30 requests per second
            return self.rest_fn(rest_url)
        else:
            print(f"limited {self.per_minute_max}/min, waiting {self.limit_expiry()}s")
            time.sleep(max(self.limit_expiry(),0.5))
            return self.push(rest_url)

    def reset_quota(self):
        self.quota = self.per_minute_max
        self.on_cooldown = False
        self.cooldown_start = None

    def limit_expiry(self):
        if self.cooldown_start:
            return max(60-(time.time()-self.cooldown_start),0)
        return 0

In [17]:
# Define tool: rest-grounding generation.
# - using gemini-2.0-flash for response generation
# - using a RAG-implementation to store groundings
# - reduce long-context by chunked pre-processing
class RestGroundingGenerator:    
    limits = None

    def __init__(self, rag_impl, with_limits: bool):
        self.rag = rag_impl
        if with_limits:
            self.limits = {}
            for rest_api in ApiLimit:
                self.limits[rest_api.value[0]] = BlockingUrlQueue(Api.get, rest_api.value[1])

    def get_limit(self, rest_api: ApiLimit) -> Optional[BlockingUrlQueue]:
        return self.limits[rest_api.value[0]] if self.limits else None

    def basemodel(self, data: str, schema: BaseModel, from_lambda: bool = False) -> Optional[BaseModel]:
        try:
            if from_lambda:
                return schema(results=json.loads(data))
            return schema.model_validate_json(data)
        except Exception as e:
            raise e

    def dailycandle(self, data: str) -> Optional[DailyCandle]:
        try:
            candle = json.loads(data)
            if "from" not in candle:
                raise ValueError("not a dailycandle / missing value for date")
            agg = self.basemodel(data, Aggregate)
            return DailyCandle(from_date=candle["from"], 
                               status=agg.status.value, 
                               symbol=agg.symbol, 
                               open=agg.open, 
                               high=agg.high, 
                               low=agg.low, 
                               close=agg.close, 
                               volume=agg.volume, 
                               otc=agg.otc, 
                               preMarket=agg.preMarket, 
                               afterHours=agg.afterHours)
        except Exception as e:
            raise e

    @retry.Retry(timeout=600)
    def try_url(self, url: str, schema: BaseModel, as_lambda: bool, with_limit: Optional[BlockingUrlQueue],
                success_fn: Callable, *args, **kwargs):
        try:
            if self.limits is None:
                data = Api.get(url)
            elif with_limit:
                data = with_limit.push(url)
            if schema is DailyCandle:
                model = self.dailycandle(data)
            else:
                model = self.basemodel(data, schema, as_lambda)
        except Exception as e:
            try:
                print(f"try_url exception: {e}")
                if issubclass(schema, RestResultPoly):
                    return success_fn(*args, **kwargs, result=self.basemodel(data, RestResultPoly))
            except Exception as not_a_result:
                print(not_a_result)
            return StopGeneration()
        else:
            return success_fn(*args, **kwargs, model=model)

    def get_symbol_matches(self, with_content, by_name: bool, model: SymbolResult):
        matches = []
        max_failed_match = model.count if not by_name else 3
        p_desc_match = 0.92
        p_symb_match = 0.95
        if model.count > 0:
            for obj in tqdm(model.get(), desc="Score similarity to query"):
                if max_failed_match > 0:
                    desc = [with_content["q"].upper(), obj.description.split("-", -1)[0]]
                    symb = [with_content["q"].upper(), obj.symbol]
                    if by_name and api.similarity(desc) > p_desc_match: 
                        matches.append(obj.symbol)
                    elif not by_name and api.similarity(symb) > p_symb_match:
                        matches.append(obj.description)
                        max_failed_match = 0
                    else:
                        max_failed_match -= 1
        if len(matches) > 0:
            self.rag.add_api_document(with_content["query"], matches, with_content["q"], "get_symbol_1")
            return matches
        return Api.Const.Stop()

    def get_quote(self, with_content, model: Quote):
        quote = model.model_dump_json()
        self.rag.add_quote_document(with_content["query"], quote, with_content["symbol"], model.t, "get_quote_1")
        return quote

    def parse_financials(self, with_content, model: BasicFinancials):
        metric = list(model.metric.items())
        chunks = []
        # Chunk the metric data.
        for i in range(0, len(metric), Api.Const.MetricBatch()):
            batch = metric[i:i + Api.Const.MetricBatch()]
            chunks.append({"question": with_content["query"], "answer": batch})
        # Chunk the series data.
        for key in model.series.keys():
            series = list(model.series[key].items())
            for s in series:
                if api.token_count(s) <= Api.Const.ChunkMax():
                    chunks.append({"question": with_content["query"], "answer": s})
                else:
                    k = s[0]
                    v = s[1]
                    for i in range(0, len(v), Api.Const.SeriesBatch()):
                        batch = v[i:i + Api.Const.SeriesBatch()]
                        chunks.append({"question": with_content["query"], "answer": {k: batch}})
        self.rag.add_rest_chunks(chunks, topic=with_content["symbol"], source="get_financials_1")
        return chunks

    def parse_news(self, with_content, model: NewsResultFinn):
        if model.count > 0:
            metas = []
            for digest in model.get():
                pub_date = datetime.fromtimestamp(digest.datetime, tz=GeneratedEvent.tz()).strftime("%Y-%m-%d")
                metas.append({"publisher": digest.source,
                              "published_est": parse(pub_date).timestamp(),
                              "news_id": digest.id,
                              "related": digest.related})
            self.rag.add_rest_chunks(model.get(), topic=with_content["symbol"], source="get_news_1",
                                     ids=[f"{digest.id}+news" for digest in model.get()],
                                     meta_opt=metas, is_update=False)
            return [digest.summary().model_dump_json() for digest in model.get()]
        return Api.Const.Stop()

    def parse_news(self, with_content, model: Optional[NewsResultPoly] = None,
                   result: Optional[RestResultPoly] = None) -> tuple[list, str]: # list of summary, next list url
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = []
            for news in model.get():
                pub_date = parse(news.published_utc).strftime("%Y-%m-%d")
                metas.append({"publisher": news.publisher.name,
                              "published_utc": parse(pub_date).timestamp(),
                              "news_id": news.id,
                              "related": json.dumps(news.tickers),
                              "keywords": json.dumps(news.keywords)})
            self.rag.add_rest_chunks(model.get(), topic=with_content["ticker"], source="get_news_2",
                                     ids=[news.id for news in model.get()],
                                     meta_opt=metas, is_update=False)
            return [news.summary().model_dump_json() for news in model.get()], model.next_url
        elif result:
            return result.model_dump_json()

    def parse_daily_candle(self, with_content, model: Optional[DailyCandle] = None,
                           result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            self.rag.add_rest_chunks(
                chunks=[model],
                topic=with_content["stocksTicker"],
                source="daily_candle_2",
                meta_opt=[{"from_date": model.from_date, "adjusted": with_content["adjusted"]}])
            return model
        elif result:
            return result

    def parse_custom_candle(self, with_content, model: Optional[CustomCandle] = None,
                            result: Optional[RestResultPoly] = None):
        if model and model.status in [RestStatus.OK, RestStatus.DELAY]:
            metas = [{
                "timespan": with_content["timespan"],
                "adjusted": with_content["adjusted"],
                "from": with_content["from"],
                "to": with_content["to"]}]*model.count
            candles = [candle.model_dump_json() for candle in model.get()]
            self.rag.add_rest_chunks(
                chunks=candles,
                topic=with_content["stocksTicker"],
                source="custom_candle_2",
                meta_opt=metas)
            return candles
        elif result:
            return result.model_dump_json()

    def parse_overview(self, with_content, model: OverviewResult):
        overview = [model.get().model_dump_json()]
        self.rag.add_rest_chunks(chunks=overview, topic=with_content["ticker"], source="ticker_overview_2")
        return overview

    def parse_trends(self, with_content, model: TrendsResult):
        if model.count > 0:
            metas = [{"period": trend.period} for trend in model.get()]
            trends = [trend.model_dump_json() for trend in model.get()]
            self.rag.add_rest_chunks(trends, topic=with_content["symbol"], source="trends_1", meta_opt=metas)
            return trends
        return Api.Const.Stop()

    def augment_market_status(self, with_id: Optional[str], model: MarketStatusResult):
        if model.get().holiday != MarketSession.NA.value:
            self.rag.set_holiday_event(model.get().exchange)
        events = self.rag.generated_events(model.get().exchange)
        model.get().session = events.session()
        model.get().isOpen = events.is_open()
        meta = {"exchange": model.get().exchange,
                "last_close": events.last_close,
                "pre_open": events.pre_open,
                "reg_open": events.reg_open,
                "reg_close": events.reg_close,
                "post_close": events.post_close,
                "timestamp": events.timestamp }
        self.rag.add_rest_chunks([model.get()],
                                 topic="market_status",
                                 source="get_market_status_1",
                                 ids=[with_id] if with_id else None,
                                 meta_opt=[meta])
        return model.get().model_dump_json()

    def get_symbol(self, content, by_name: bool = True):
        return self.try_url(
            f"https://finnhub.io/api/v1/search?q={content['q']}&exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=SymbolResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_symbol_matches,
            with_content=content,
            by_name=by_name)

    def get_current_price(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/quote?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=Quote,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.get_quote,
            with_content=content)

    def get_market_status(self, content, store_id: Optional[str] = None):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/market-status?exchange={content['exchange']}&token={FINNHUB_API_KEY}",
            schema=MarketStatusResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.augment_market_status,
            with_id=store_id)

    def get_peers(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/peers?symbol={content['symbol']}&grouping={content['grouping']}&token={FINNHUB_API_KEY}",
            schema=PeersResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=lambda model: model)

    def get_basic_financials(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/metric?symbol={content['symbol']}&metric={content['metric']}&token={FINNHUB_API_KEY}",
            schema=BasicFinancials,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_financials,
            with_content=content)

    def get_news_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/company-news?symbol={content['symbol']}&from={content['from']}&to={content['to']}&token={FINNHUB_API_KEY}",
            schema=NewsResultFinn,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_news,
            with_content=content)

    def get_news_tagged(self, content):
        next_url = f"https://api.polygon.io/v2/reference/news?ticker={content['ticker']}&published_utc.gte={content['published_utc.gte']}&published_utc.lte={content['published_utc.lte']}&order={content['order']}&limit={content['limit']}&sort={content['sort']}&apiKey={POLYGON_API_KEY}"
        news = []
        while True:
            news_list, next_url = self.try_url(
                next_url,
                schema=NewsResultPoly,
                as_lambda=False,
                with_limit=self.get_limit(ApiLimit.POLY),
                success_fn=self.parse_news,
                with_content=content)
            news += news_list
            if next_url is None:
                break
            next_url += f"&apiKey={POLYGON_API_KEY}"
        return news

    def get_daily_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v1/open-close/{content['stocksTicker']}/{content['date']}?adjusted={content['adjusted']}&apiKey={POLYGON_API_KEY}",
            schema=DailyCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_daily_candle,
            with_content=content)

    def get_custom_candle(self, content):
        return self.try_url(
            f"https://api.polygon.io/v2/aggs/ticker/{content['stocksTicker']}/range/{content['multiplier']}/{content['timespan']}/{content['from']}/{content['to']}?adjusted={content['adjusted']}&sort={content['sort']}&limit={content['limit']}&apiKey={POLYGON_API_KEY}",
            schema=CustomCandle,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_custom_candle,
            with_content=content)

    def get_overview(self, content):
        return self.try_url(
            f"https://api.polygon.io/v3/reference/tickers/{content['ticker']}?apiKey={POLYGON_API_KEY}",
            schema=OverviewResult,
            as_lambda=False,
            with_limit=self.get_limit(ApiLimit.POLY),
            success_fn=self.parse_overview,
            with_content=content)

    def get_trends_simple(self, content):
        return self.try_url(
            f"https://finnhub.io/api/v1/stock/recommendation?symbol={content['symbol']}&token={FINNHUB_API_KEY}",
            schema=TrendsResult,
            as_lambda=True,
            with_limit=self.get_limit(ApiLimit.FINN),
            success_fn=self.parse_trends,
            with_content=content)

### Callable Functions

In [18]:
# Callable functions in openapi schema.
decl_get_symbol_1 = types.FunctionDeclaration(
    name="get_symbol_1",
    description="""Search for the stock ticker symbol of a given company, security, isin or cusip. Each ticker
                   entry provides a description, symbol, and asset type. If this doesn't help you should try 
                   calling get_wiki_tool_response next.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """A ticker symbol to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["q", "exchange", "query"]
    }
)

decl_get_symbols_1 = types.FunctionDeclaration(
    name="get_symbols_1",
    description="""List all supported symbols and tickers. The results are filtered by exchange code.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter the results."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["exchange", "query"]
    }
)

decl_get_name_1 = types.FunctionDeclaration(
    name="get_name_1",
    description="""Search for the name associated with a stock ticker or symbol's company, security, isin or cusip. 
    Each ticker entry provides a description, matching symbol, and asset type.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": """The symbol or ticker to search for."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "company": {
                "type": "string",
                "description": "The company you're searching for."
            }
        },
        "required": ["q", "exchange", "query", "company"]
    }
)

decl_get_symbol_quote_1 = types.FunctionDeclaration(
    name="get_symbol_quote_1",
    description="""Search for the current price or quote of a stock ticker or symbol. The response is
                   provided in json format. Each response contains the following key-value pairs:
                   
                   c: Current price,
                   d: Change,
                  dp: Percent change,
                   h: High price of the day,
                   l: Low price of the day,
                   o: Open price of the day,
                  pc: Previous close price,
                   t: Epoch timestamp of price in seconds.

                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol for a company, security, isin, or cusip." 
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            },
            "exchange": {
                "type": "string",
                "description": "The exchange code used to filter quotes. This must always be 'US'."
            }
        },
        "required": ["symbol", "query", "exchange"]
    }
)

decl_get_local_datetime = types.FunctionDeclaration(
    name="get_local_datetime",
    description="""Converts an array of timestamps from epoch time to the local timezone format. The result is an array
                   of date and time in locale appropriate format. Suitable for use in a locale appropriate response.
                   Treat this function as a vector function. Always prefer to batch timestamps for conversion. Use this
                   function to format date and time in your responses.""",
    parameters={
        "type": "object",
        "properties": {
            "t": {
                "type": "array",
                "description": """An array of timestamps in seconds since epoch to be converted. The order of
                                  timestamps matches the order of conversion.""",
                "items": {
                    "type": "integer"
                }
            }
        },
        "required": ["t"]
    }
)

decl_get_market_status_1 = types.FunctionDeclaration(
    name="get_market_status_1",
    description="""Get the current market status of global exchanges. Includes whether exchanges are open or closed.  
                   Also includes holiday details if applicable. The response is provided in json format. Each response 
                   contains the following key-value pairs:

                   exchange: Exchange code,
                   timezone: Timezone of the exchange,
                    holiday: Holiday event name, or null if it's not a holiday,
                     isOpen: Whether the market is open at the moment,
                          t: Epoch timestamp of status in seconds (Eastern Time),
                    session: The market session can be 1 of the following values: 
                    
                    pre-market,regular,post-market when open, or null if closed.
                    
                    Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_market_session_1 = types.FunctionDeclaration(
    name="get_market_session_1",
    description="Get the current market session of global exchanges.",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_company_peers_1 = types.FunctionDeclaration(
    name="get_company_peers_1",
    description="""Search for a company's peers. Returns a list of peers operating in the same country and in the same
                   sector, industry, or subIndustry. Each response contains the following key-value pairs: 
                   
                   symbol: The company's stock ticker symbol, 
                   peers: A list containing the peers.
                   
                   Each peers entry contains the following key-value pairs:
                   
                   symbol: The peer company's stock ticker symbol, 
                   name: The peer company's name.
                   
                   Parse the response and respond according to this information.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "The stock ticker symbol of a company to obtain peers."
            },
            "grouping": {
                "type": "string",
                "description": """This parameter may be one of the following values: sector, industry, subIndustry.
                                  Always use subIndustry unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "grouping", "exchange", "query"]
    }
)

decl_get_exchange_codes_1 = types.FunctionDeclaration(
    name="get_exchange_codes_1",
    description="""Get a dictionary mapping all supported exchange codes to their names."""
)

decl_get_exchange_code_1 = types.FunctionDeclaration(
    name="get_exchange_code_1",
    description="""Search for the exchange code to use when filtering by exchange. The result will be one or
                   more exchange codes provided as a comma-separated string value.""",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "Specifies which exchange code to search for."
            }
        },
        "required": ["q"]
    }
)

decl_get_financials_1 = types.FunctionDeclaration(
    name="get_financials_1",
    description="""Get company basic financials such as margin, P/E ratio, 52-week high/low, etc. Parse the response for 
                   key-value pairs in json format and interpret their meaning as stock market financial indicators.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "metric": {
                "type": "string",
                "description": "It must always be declared as the value 'all'"
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "metric", "query"]
    }
)

decl_get_daily_candlestick_2 = types.FunctionDeclaration(
    name="get_daily_candlestick_2",
    description="""Get a historical daily stock ticker candlestick / aggregate bar (OHLC). 
                   Includes historical daily open, high, low, and close prices. Also includes historical daily trade
                   volume and pre-market/after-hours trade prices. It provides the last trading days' data after 
                   11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "date": {
                "type": "string",
                "format": "date-time",
                "description": """The date of the requested candlestick in format YYYY-MM-DD."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "date", "adjusted", "exchange", "query"]
    },
)

decl_get_company_news_1 = types.FunctionDeclaration(
    name="get_company_news_1",
    description="Retrieve the most recent news articles related to a specified ticker.",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company.",
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD. It must be more recent than the parameter 'from'. The
                                  default value is today's date."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "from", "to", "query"]
    },
)

decl_get_custom_candlestick_2 = types.FunctionDeclaration(
    name="get_custom_candlestick_2",
    description="""Get a historical stock ticker candlestick / aggregate bar (OHLC) over a custom date range and 
                   time interval in Eastern Time. Includes historical open, high, low, and close prices. Also 
                   includes historical daily trade volume and pre-market/after-hours trade prices. It includes 
                   the last trading days' data after 11:59PM Eastern Time.""",
    parameters={
        "type": "object",
        "properties": {
            "stocksTicker": {
                "type": "string",
                "description": "The stock ticker symbol of a company to search for.",
            },
            "multiplier": {
                "type": "integer",
                "description": "This must be included and equal to 1 unless told otherwise."
            },
            "timespan": {
                "type": "string",
                "description": """The size of the candlestick's time window. This is allowed to be one of the following:
                                  second, minute, hour, day, week, month, quarter, or year. The default value is day."""
            },
            "from": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'to'."""
            },
            "to": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'from'. The 
                                  default is one weekday before get_last_market_close.
                                  Replace more recent dates with the default."""
            },
            "adjusted": {
                "type": "string",
                "description": """May be true or false. Indicates if the results should be adjusted for splits.
                                  Use true unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """This must be included. May be one of asc or desc. asc will sort by timestmap in 
                                  ascending order. desc will sort by timestamp in descending order."""
            },
            "limit": {
                "type": "integer",
                "description": """Set the number of base aggregates used to create this candlestick. This must be 5000 
                                  unless told to limit base aggregates to something else."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["stocksTicker", "multiplier", "timespan", "from", "to", "adjusted", "sort", "limit", "query"]
    },
)

decl_get_last_market_close = types.FunctionDeclaration(
    name="get_last_market_close",
    description="""Get the last market close of the specified exchange in Eastern Time. The response has already
                   been converted by get_local_datetime so this step should be skipped.""",
    parameters={
        "type": "object",
        "properties": {
            "exchange": {
                "type": "string",
                "description": """The exchange code used to filter results. When not specified the default exchange 
                                  code you should use is 'US' for the US exchanges. A dictionary mapping all supported 
                                  exchange codes to their names be retrieved by calling get_exchange_codes_1. 
                                  Search for an exchange code to use by calling get_exchange_code_1, specifying the
                                  exchange code to search for."""
            }
        },
        "required": ["exchange"]
    }
)

decl_get_ticker_overview_2 = types.FunctionDeclaration(
    name="get_ticker_overview_2",
    description="""Retrieve comprehensive details for a single ticker symbol. It's a deep look into a company’s 
    fundamental attributes, including its primary exchange, standardized identifiers (CIK, composite FIGI, 
    share class FIGI), market capitalization, industry classification, and key dates. Also includes branding assets in
    the form of icons and logos.
    """,
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol of a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["ticker", "query"]
    }
)

decl_get_recommendation_trends_1 = types.FunctionDeclaration(
    name="get_recommendation_trends_1",
    description="""Get the latest analyst recommendation trends for a company.
                The data includes the latest recommendations as well as historical
                recommendation data for each month. The data is classified according
                to these categories: strongBuy, buy, hold, sell, and strongSell.
                The date of a recommendation indicated by the value of 'period'.""",
    parameters={
        "type": "object",
        "properties": {
            "symbol": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["symbol", "query"]
    }
)

decl_get_news_with_sentiment_2 = types.FunctionDeclaration(
    name="get_news_with_sentiment_2",
    description="""Retrieve the most recent news articles related to a specified ticker. Each article includes 
                   comprehensive coverage. Including a summary, publisher information, article metadata, 
                   and sentiment analysis.""",
    parameters={
        "type": "object",
        "properties": {
            "ticker": {
                "type": "string",
                "description": "Stock ticker symbol for a company."
            },
            "published_utc.gte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be older than the parameter 'published_utc.lte'. 
                                  The default value is one-month ago from today's date."""
            },
            "published_utc.lte": {
                "type": "string",
                "format": "date-time",
                "description": """A date in format YYYY-MM-DD must be more recent than the parameter 'published_utc.gte'.
                                  The default is one weekday prior to get_last_market_close (excluding weekends).
                                  Replace more recent dates with the default."""
            },
            "order": {
                "type": "string",
                "description": """Must be desc for descending order, or asc for ascending order.
                                  When order is not specified the default is descending order.
                                  Ordering will be based on the parameter 'sort'."""
            },
            "limit": {
                "type": "integer",
                "description": """This must be included and equal to 1000 unless told otherwise."""
            },
            "sort": {
                "type": "string",
                "description": """The sort field used for ordering. This value must
                                  always be published_utc."""
            },
            "query": {
                "type": "string",
                "description": "The question you're attempting to answer."
            }
        },
        "required": ["limit", "ticker", "published_utc.gte", "published_utc.lte", "order", "sort", "query"]
    }
)

decl_get_rag_tool_response = types.FunctionDeclaration(
    name="get_rag_tool_response",
    description="""A database containing useful financial information. Always check here for answers first.""",
    parameters={
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "A question needing an answer. Asked as a simple string."
            }
        }
    }
)

decl_get_wiki_tool_response = types.FunctionDeclaration(
    name="get_wiki_tool_response",
    description="""Answers questions that still have unknown answers. Retrieve a wiki page related to a company, 
                   product, or service. Each web page includes detailed company information, financial indicators, 
                   tickers, symbols, history, and products and services.""",
    parameters={
        "type": "object",
        "properties": {
            "id": {
                "type": "string",
                "description": "The question's company or product. Just the name and no other details."
            },
            "q": {
                "type": "string",
                "description": "The complete, unaltered, query string."
            }
        },
        "required": ["id", "q"]
    }
)

decl_get_search_tool_response = types.FunctionDeclaration(
    name="get_search_tool_response",
    description="Answers questions that still have unknown answers. Use it after checking all your other tools.",
    parameters={
        "type": "object",
        "properties": {
            "q": {
                "type": "string",
                "description": "The question needing an answer. Asked as a simple string."
            },
            "id": {
                "type": "string",
                "description": "The question's company or product. In one word. Just the name and no other details."
            }
        },
        "required": ["q", "id"]
    }
)

In [19]:
# Import the finance api secret keys.

POLYGON_API_KEY = UserSecretsClient().get_secret("POLYGON_API_KEY")
FINNHUB_API_KEY = UserSecretsClient().get_secret("FINNHUB_API_KEY")

In [20]:
# Instantiate tools and load the exchange data from source csv.
# - Identifies exchanges by a 1-2 letter code which can be used to filter response data.
# - Also maps the exchange code to exchange details.
try:
    df = pandas.read_csv("/kaggle/input/exchanges/exchanges_src.csv")
except FileNotFoundError as e:
    df = pandas.read_csv("exchanges_src.csv") # local run
df = df.drop(["close_date"], axis=1).fillna("")
df.to_csv("exchanges.csv", index=False)
exchanges = CSVLoader(file_path="exchanges.csv", encoding="utf-8", csv_args={"delimiter": ","}).load()

# Prepare a RAG tool for use and add the exchange data.
tool_rag = RetrievalAugmentedGenerator(api.args.CLIENT, "finance")
tool_rag.add_documents_list(exchanges)

# Prepare a the grounding tools for use.
tool_wiki = WikiGroundingGenerator(api.args.CLIENT, tool_rag)
tool_ground = SearchGroundingGenerator(api.args.CLIENT, tool_rag)
tool_rest = RestGroundingGenerator(tool_rag, with_limits=True)

Generate document embedding: 0it [00:00, ?it/s]


### Function Calling Expert

In [21]:
# Implement the callable functions and function handler.

def ask_rag_tool(content):
    return tool_rag.generate_answer(content["question"]).text

def ask_wiki_tool(content):
    return tool_wiki.generate_answer(content["q"], content["id"])

def ask_search_tool(content):
    return tool_ground.generate_answer(content["q"], content["id"])

def get_exchange_codes_1(content):
    return tool_rag.get_exchange_codes()

def get_exchange_code_1(content):
    return tool_rag.get_exchange_codes(with_query=content)
    
def last_market_close(content):
    return tool_rag.last_market_close(content["exchange"])
    
def get_symbol_1(content, by_name: bool = True):
    stored = tool_rag.get_api_documents(content["query"], content["q"], "get_symbol_1")
    if len(stored) == 0:
        return tool_rest.get_symbol(content, by_name)
    return json.loads(stored[0].docs)

def get_symbols_1(content):
    return None # todo

def get_name_1(content):
    return get_symbol_1(content, by_name = False)

def get_quote_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_quote_1")
    if tool_rag.generated_events(content["exchange"]).is_open():
        return get_current_price_1(content)
    elif len(stored) > 0:
        last_close = parse(tool_rag.last_market_close(content["exchange"])).timestamp()
        for quote in stored:
            if quote.meta["timestamp"] >= last_close:
                return [quote.docs for quote in stored]
    return get_current_price_1(content)

def get_current_price_1(content):
    return tool_rest.get_current_price(content)

def get_market_status_1(content):
    stored, has_update = tool_rag.get_market_status(content['exchange'])
    if has_update:
        with_id = stored[0].store_id if len(stored) > 0 else None
        return tool_rest.get_market_status(content, with_id)
    return stored[0].docs

def get_session_1(content):
    return json.loads(get_market_status_1(content))["session"]

def get_peers_1(content):
    stored = tool_rag.get_peers_document(content["query"], content["symbol"], content['grouping'])
    if len(stored) == 0:
        peers = tool_rest.get_peers(content)
        if peers.count > 0:
            names = []
            for peer in peers.get():
                if peer == content["symbol"]:
                    continue # skip including the query symbol in peers
                name = get_name_1(dict(q=peer, exchange=content["exchange"], query=content["query"]))
                if name != Api.Const.Stop():
                    data = {"symbol": peer, "name": name}
                    names.append(data)
            tool_rag.add_peers_document(content["query"], names, content["symbol"], "get_peers_1", content['grouping'])
            return names
        return Api.Const.Stop()
    return json.loads(stored[0].docs)["peers"]

def local_datetime(content):
    local_t = []
    for timestamp in content["t"]:
        local_t.append(local_date_from_epoch(timestamp))
    return local_t

def local_date_from_epoch(timestamp):
    if len(str(timestamp)) == 13:
        return datetime.fromtimestamp(timestamp/1000, tz=GeneratedEvent.tz()).strftime('%c')
    else:
        return datetime.fromtimestamp(timestamp, tz=GeneratedEvent.tz()).strftime('%c')

def get_financials_1(content):
    stored = tool_rag.get_basic_financials(content["query"], content["symbol"], "get_financials_1")
    if len(stored) == 0:
        return tool_rest.get_basic_financials(content)
    return [chunk.docs for chunk in stored]

def get_news_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "get_news_1")
    if len(stored) == 0:
        return tool_rest.get_news_simple(content)
    return [NewsTypeFinn.model_validate_json(news.docs).summary().model_dump_json() for news in stored]

def get_daily_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="daily_candle_2", 
        meta_opt=[{"from_date": content["date"], "adjusted": content["adjusted"]}])
    if len(stored) == 0:
        candle = tool_rest.get_daily_candle(content)
        # Attempt to recover from choosing a holiday.
        candle_date = parse(content["date"])
        if candle.status is RestStatus.NONE and candle_date.weekday() == 0 or candle_date.weekday() == 4:
            if candle_date.weekday() == 0: # index 0 is monday, index 4 is friday
                content["date"] = candle_date.replace(day=candle_date.day-3).strftime("%Y-%m-%d")
            else:
                content["date"] = candle_date.replace(day=candle_date.day-1).strftime("%Y-%m-%d")
            return get_daily_candle_2(content)
        return candle.model_dump_json()
    return [json.loads(candle.docs) for candle in stored]

def get_custom_candle_2(content):
    stored = tool_rag.get_api_documents(
        query=content["query"], topic=content["stocksTicker"], source="custom_candle_2", 
        meta_opt=[{
            "timespan": content["timespan"],
            "adjusted": content["adjusted"],
            "from": content["from"],
            "to": content["to"]}])
    if len(stored) == 0:
        return tool_rest.get_custom_candle(content)
    return [json.loads(candle.docs) for candle in stored]

def get_overview_2(content):
    stored = tool_rag.get_api_documents(content["query"], content["ticker"], "ticker_overview_2")
    if len(stored) == 0:
        return tool_rest.get_overview(content)
    return json.loads(stored[0].docs)

def get_trends_1(content):
    stored = tool_rag.get_api_documents(content["query"], content["symbol"], "trends_1")
    if len(stored) == 0:
        return tool_rest.get_trends_simple(content)
    return [json.loads(trend.docs) for trend in stored]

def get_news_2(content):
    timestamp_from = parse(content["published_utc.gte"]).timestamp()
    timestamp_to = parse(content["published_utc.lte"]).timestamp()
    news_from = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_from}])
    news_to = tool_rag.get_api_documents(
        content["query"], content["ticker"], "get_news_2", [{"published_utc": timestamp_to}])
    if len(news_from) > 0 and len(news_to) > 0:
        stored = tool_rag.get_api_documents(
            content["query"], content["ticker"], "get_news_2",
            [{"published_utc": {"$gte": timestamp_from}},
             {"published_utc": {"$lte": timestamp_to}}])
        return [NewsTypePoly.model_validate_json(news.docs).summary().model_dump_json() for news in stored]
    return tool_rest.get_news_tagged(content)
        
finance_tool = types.Tool(
    function_declarations=[
        decl_get_symbol_1,
        decl_get_symbols_1,
        decl_get_name_1,
        decl_get_symbol_quote_1,
        decl_get_market_status_1,
        decl_get_market_session_1,
        decl_get_company_peers_1,
        decl_get_local_datetime,
        decl_get_last_market_close,
        decl_get_exchange_codes_1,
        decl_get_exchange_code_1,
        decl_get_financials_1,
        decl_get_daily_candlestick_2,
        decl_get_custom_candlestick_2,
        decl_get_ticker_overview_2,
        decl_get_recommendation_trends_1,
        decl_get_news_with_sentiment_2,
        decl_get_rag_tool_response,
        decl_get_wiki_tool_response,
        decl_get_search_tool_response
    ]
)

function_handler = {
    "get_symbol_1": get_symbol_1,
    "get_symbols_1": get_symbols_1,
    "get_name_1": get_name_1,
    "get_symbol_quote_1": get_quote_1,
    "get_market_status_1": get_market_status_1,
    "get_market_session_1": get_session_1,
    "get_company_peers_1": get_peers_1,
    "get_local_datetime": local_datetime,
    "get_last_market_close": last_market_close,
    "get_exchange_codes_1": get_exchange_codes_1,
    "get_exchange_code_1": get_exchange_code_1,
    "get_financials_1": get_financials_1,
    "get_daily_candlestick_2": get_daily_candle_2,
    "get_custom_candlestick_2": get_custom_candle_2,
    "get_ticker_overview_2": get_overview_2,
    "get_recommendation_trends_1": get_trends_1,
    "get_news_with_sentiment_2": get_news_2,
    "get_rag_tool_response": ask_rag_tool,
    "get_wiki_tool_response": ask_wiki_tool,
    "get_search_tool_response": ask_search_tool
}

In [22]:
# Implement the function calling expert.
# Define the system prompt.
instruction = f"""You are a helpful and informative bot that answers finance and stock market questions. 
Only answer the question asked and do not change topic. While the answer is still
unknown you must follow these rules for predicting function call order:

RULE#1: Always consult your other functions before get_search_tool_response.
RULE#2: Always consult get_wiki_tool_response before get_search_tool_response.
RULE#3: Always consult get_search_tool_response last.
RULE#4: Always convert timestamps with get_local_datetime and use the converted date/time in your response.
RULE#5: Always incorporate as much useful information from tools and functions in your response."""

def get_response():
    # Enable system prompt, function calling and minimum-randomness.
    config_fncall = types.GenerateContentConfig(
        system_instruction=instruction,
        tools=[finance_tool],
        temperature=0.0
    )
    memory.response = api.retriable(
        api.args.CLIENT.models.generate_content,
        model=api(Api.Model.GEN),
        config=config_fncall,
        contents=memory.contents)

def retry_last_send():
    api.generation_fail()
    time.sleep(api.dt_between)
    get_response()

@retry.Retry(
    predicate=is_retriable,
    initial=2.0,
    maximum=64.0,
    multiplier=2.0,
    timeout=600,
)
def send_message(prompt):
    #display(Markdown("#### Prompt"))
    #print(prompt, "\n")
    memory.set_prompt(prompt)
    # Handle cases with multiple chained function calls.
    function_calling_in_process = True
    # Send the initial user prompt and function declarations.
    get_response()
    while function_calling_in_process:
        try:
            response_parts = memory.response.candidates[0].content.parts
            # A summary response never includes function calls.
            if not any(part.function_call for part in response_parts):
                memory.set_summary("\n".join(e.text for e in response_parts))
                function_calling_in_process = False
                break # The function calling chain is complete.
            else:
                # A part can be a function call or reasoning-step.
                for part in response_parts:
                    if function_call := part.function_call:
                        # Extract the function call.
                        fn_name = function_call.name
                        #display(Markdown("#### Predicted function name"))
                        #print(fn_name, "\n")
                        # Extract the function call arguments.
                        fn_args = {key: value for key, value in function_call.args.items()}
                        #display(Markdown("#### Predicted function arguments"))
                        #print(fn_args, "\n")
                        # Call the predicted function.
                        print("send_message: get function response")
                        api_response = function_handler[fn_name](fn_args)[:20000] # Stay within the input token limit
                        #display(Markdown("#### API response"))
                        #print(api_response[:500], "...", "\n")
                        # Create an API response part.
                        api_response_part = types.Part.from_function_response(
                            name=fn_name,
                            response={"content": api_response},
                        )
                        memory.update_contents(function_call, api_response_part)
                    else:
                        #display(Markdown("#### Natural language reasoning step"))
                        #print(part.text)
                        memory.set_reason(part.text)
                print("send_message: updating state")
                get_response() # Send the updated prompt.
                print("send_message: got a response")
        except Exception as e:
            if isinstance(response_parts, list):
                print("send_message: generated wrong function arguments")
            retry_last_send()
            
    # Show the final natural language summary.
    display(Markdown("#### Natural language response"))
    display(Markdown(memory.summary))

## RAG Baseline Check

In [23]:
response = tool_rag.get_exchanges_csv(
    """Give me a dictionary in string form. It must contain key:value pairs mapping 
    exchange code to name. Just the dictionary string in pretty form.""")
print(response.candidates[0].content.parts[-1].text)

response = tool_rag.get_exchanges_csv(
    """What is the Germany exchange code? Return only the exchange codes as a simple 
    comma separated value that I can copy.""")
print(response.candidates[0].content.parts[-1].text, "\n")

response = tool_rag.get_exchanges_csv("What are the Germany exchanges and thier corresponding exchange codes?")
print(response.text, "\n")

response = tool_rag.generate_answer("What are Google's stock ticker symbols?")
print(response.text, "\n")

response = tool_rag.generate_answer("What is Facebook's stock ticker symbol?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv("What are the US exchange operating hours?")
print(response.text, "\n")

response = tool_rag.get_exchanges_csv(
    f"""Answer based on your knowledge of exchange operating hours.
    Do not answer in full sentences. Omit all chat and provide the answer only.
    The fields pre_market and post_market both represent extended operating hours.

    The current date and time: {datetime.now(GeneratedEvent.tz()).strftime('%c')}

    Weekdays are: Mon, Tue, Wed, Thu, Fri.
    On weekdays all exchanges open after pre-market and regular hours.
    On weekdays all exchanges close after regular and post-market hours.
    
    Weekends are: Sat, Sun.
    Always exclude weekends from exchange operating hours.
    A list of holidays in date format mm-dd-yyyy: {tool_rag.holidays["US"]}
    Always exclude holidays from exchange operating hours.
    When the answer is a holiday use the prior weekday for close.
    When the answer is a holiday use the next weekday for open.
    
    Consider the US exchange's operating hours.
    Provide the most recent weekday's close including post_market hours.
    
    Answer with a date that uses this format: '%a %b %d %X %Y'.""")
print(response.candidates[0].content.parts[-1].text)

```
{
    "VN": "Vietnam exchanges including HOSE, HNX and UPCOM",
    "AD": "ABU DHABI SECURITIES EXCHANGE",
    "US": "US exchanges (NYSE, Nasdaq)",
    "CO": "OMX NORDIC EXCHANGE COPENHAGEN A/S",
    "QA": "QATAR EXCHANGE",
    "BA": "BOLSA DE COMERCIO DE BUENOS AIRES",
    "MX": "BOLSA MEXICANA DE VALORES (MEXICAN STOCK EXCHANGE)",
    "PR": "PRAGUE STOCK EXCHANGE",
    "HK": "HONG KONG EXCHANGES AND CLEARING LTD",
    "CA": "Egyptian Stock Exchange",
    "AX": "ASX - ALL MARKETS",
    "SX": "DEUTSCHE BOERSE Stoxx",
    "KQ": "KOREA EXCHANGE (KOSDAQ)",
    "DB": "DUBAI FINANCIAL MARKET",
    "PM": "Philippine Stock Exchange",
    "KS": "KOREA EXCHANGE (STOCK MARKET)",
    "ST": "NASDAQ OMX NORDIC STOCKHOLM",
    "DU": "BOERSE DUESSELDORF",
    "TL": "NASDAQ OMX TALLINN",
    "AT": "ATHENS EXCHANGE S.A. CASH MARKET",
    "SW": "SWISS EXCHANGE",
    "LS": "NYSE EURONEXT - EURONEXT LISBON",
    "SI": "SINGAPORE EXCHANGE",
    "RG": "NASDAQ OMX RIGA",
    "CR": "CARACAS STOCK EXCHANGE"

## SC1 Baseline Check

In [24]:
# Wait 59s for rate-limits to reset on FREE-tier.
if api.args.API_LIMIT is Api.Limit.FREE.value:
    print("Gemini API limit is FREE. Waiting 59s...")
    time.sleep(59)

Gemini API limit is FREE. Waiting 59s...
Api.refill_rpm 10


In [25]:
send_message("What is the current session for US exchanges?")

send_message: get function response


Generate US->MarketEvent.POST_CLOSE: 100%|██████████| 1/1 [00:02<00:00,  2.54s/it]
Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

The current session for US exchanges is regular.

In [26]:
send_message("What is the US market status?")

Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The US market is currently **open** for the **regular** session.

The current market status was last updated on **Mon Dec 1 11:35:58 2025** (Eastern Time). There are no holidays affecting the market today.

In [27]:
send_message("When was the last US market close?")

send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

The last US market close was on Friday, November 28, 2025, at 8:00:00 PM.

In [28]:
send_message("What is Apple's stock ticker?")

send_message: get function response


Score similarity to query: 100%|██████████| 10/10 [00:01<00:00,  8.42it/s]
Generate api embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Apple's stock ticker symbol is AAPL.

In [29]:
send_message("What is the current price of Amazon stock? Display the result as a json string in markdown.")

send_message: get function response


Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  3.43it/s]
Generate api embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: got a response
send_message: get function response


Generate quote embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

```json
{
"c": 234.125,
"d": 0.905,
"dp": 0.388,
"h": 235.797,
"l": 231.88,
"o": 231.88,
"pc": 233.22,
"t": 1764606938
}
```

In [30]:
send_message("""Show me Apple's basic financials and help me understand key performance metrics. 
How has the stock performed?""")

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's an overview of Apple's financials and stock performance based on the data from 2025:

**Financial Highlights:**

*   **Revenue Growth:** Apple has demonstrated strong revenue growth with a TTM YoY growth of 6.43%.
*   **Profitability:** The company maintains a high level of profitability, with a TTM Net Profit Margin of 26.92%.
*   **Gross Margin:** Apple's gross margin is also strong, with a TTM of 46.91%.
*   **Earnings Per Share (EPS):** Apple's EPS has grown significantly, with a TTM EPS Excl. Extra Items of 7.4593, reflecting a growth of 22.89% YoY.
*   **P/E Ratio:** The trailing twelve months (TTM) Price-to-Earnings (P/E) ratio is 36.7859. The forward P/E ratio is 33.6884.
*   **Dividends:** Apple pays a dividend, with a current dividend yield of 0.3743%. The dividend per share TTM is $1.0318.
*   **Return on Equity (ROE):** Apple's ROE is very high, with a TTM value of 164.05%.
*   **52-Week Performance:** The 52 week high is 280.38, reached on 2025-11-25, and the 52 week low is 169.2101, reached on 2025-04-08.
*   **Stock Performance:**
    *   The stock has a 5-Day Price Return Daily of 2.711%.
    *   The Month-to-Date Price Return Daily is 3.1364%.
    *   The Year-to-Date Price Return Daily is 11.3529%.
    *   The 52-Week Price Return Daily is 18.6293%.
    *   The 13-Week Price Return Daily is 19.9045%.
    *   The 26-Week Price Return Daily is 39.2788%.

**Key Performance Metrics:**

*   **Beta:** The beta is 1.0957, indicating that the stock is slightly more volatile than the market.
*   **PEG Ratio:** The PEG ratio is 1.6255, which is above 1, suggesting that the stock may be overvalued relative to its earnings growth.

**Financial Health:**

*   **Current Ratio:** The current ratio is 0.8933, which is below 1, suggesting that the company may have some liquidity issues.
*   **Debt-to-Equity Ratio:** The Long Term Debt to Equity Annual is 1.0623, indicating a moderate level of debt relative to equity.

**Valuation:**

*   **Price-to-Book Ratio (P/B):** The P/B ratio is 55.8825, which is very high, suggesting that the stock may be overvalued.
*   **Price-to-Sales Ratio (P/S):** The P/S ratio is 9.9009.

**Additional Indicators:**

*   **Price Relative to S&P 500:**
    *   Price Relative to S&P 500 13 Week: 14.5926
*   **Price Relative to S&P 500 26 Week:** 23.6753
*   **Price Relative to S&P 500 52 Week:** 4.8542



In [31]:
send_message("I need Apple's daily candlestick from 2025-05-05")

send_message: get function response
Api.zero_error: model is now gemini-2.5-flash


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here is Apple's daily candlestick data for 2025-05-05:

*   **Symbol:** AAPL
*   **Open:** 203.1
*   **High:** 204.1
*   **Low:** 198.21
*   **Close:** 198.89
*   **Volume:** 69018452
*   **Pre-market:** 205.0
*   **After-hours:** 198.6

In [32]:
send_message("Tell me who are Apple's peers?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 17.55it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.47it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.60it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 16.66it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.33it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.52it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  7.18it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.50it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  6.80it/s]

send_message: updating state
send_message: got a response


#### Natural language response

Apple's peers are: DELL TECHNOLOGIES -C (DELL), WESTERN DIGITAL CORP (WDC), SANDISK CORP (SNDK), HEWLETT PACKARD ENTERPRISE (HPE), PURE STORAGE INC - CLASS A (PSTG), HP INC (HPQ), NETAPP INC (NTAP), SUPER MICRO COMPUTER INC (SMCI), IONQ INC (IONQ), QUANTUM COMPUTING INC (QUBT), COMPOSECURE INC-A (CMPO).

In [33]:
send_message("Tell me who are Amazon's peers?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.42it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  6.81it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.53it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 5/5 [00:00<00:00, 16.66it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 11/11 [00:00<00:00, 35.22it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 2/2 [00:00<00:00,  6.88it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.48it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.18it/s]
Generate api embedding: 0it [00:00, ?it/s]
Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.60it/

send_message: updating state
send_message: got a response


#### Natural language response

Amazon's peers include Coupang Inc (CPNG), eBay Inc (EBAY), Dillard's Inc-CL A (DDS), Ollie's Bargain Outlet Holdings (OLLI), Macy's Inc (M), Etsy Inc (ETSY), Kohl's Corp (KSS), Pattern Group Inc-CL A (PTRN), Savers Value Village Inc (SVV), and Groupon Inc (GRPN).

In [34]:
send_message("""Locate Apple's stock ticker, then download recommendation trends of all Apple's peers by sub-industry, 
and then finally compare them.""")

Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


...

send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

Apple's stock ticker is **AAPL**.

Here is a comparison of the latest analyst recommendation trends for Apple and its peers in the sub-industry, based on data as of **Sun Dec 1 00:00:00 2024**:

| Company (Ticker) | Strong Buy | Buy | Hold | Sell | Strong Sell | Total Recommendations |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **Apple (AAPL)** | 15 | 23 | 16 | 2 | 0 | 56 |
| **DELL TECHNOLOGIES -C (DELL)** | 8 | 16 | 7 | 1 | 0 | 32 |
| **WESTERN DIGITAL CORP (WDC)** | 6 | 19 | 6 | 0 | 0 | 31 |
| **SANDISK CORP (SNDK)** | 7 | 11 | 6 | 0 | 0 | 24 |
| **HEWLETT PACKARD ENTERPRISE (HPE)** | 6 | 7 | 13 | 0 | 0 | 26 |
| **PURE STORAGE INC - CLASS A (PSTG)** | 7 | 13 | 6 | 1 | 0 | 27 |
| **HP INC (HPQ)** | 1 | 1 | 16 | 6 | 1 | 25 |
| **NETAPP INC (NTAP)** | 3 | 10 | 14 | 0 | 0 | 27 |
| **SUPER MICRO COMPUTER INC (SMCI)** | 4 | 10 | 9 | 3 | 0 | 26 |
| **IONQ INC (IONQ)** | 2 | 10 | 4 | 0 | 0 | 16 |
| **QUANTUM COMPUTING INC (QUBT)** | 2 | 5 | 2 | 0 | 0 | 9 |
| **COMPOSECURE INC-A (CMPO)** | 2 | 8 | 2 | 0 | 0 | 12 |

### Key Observations:

*   **Apple (AAPL)** has the highest number of total recommendations (56) and a strong overall positive sentiment, with 38 "Buy" or "Strong Buy" ratings compared to 18 "Hold" or "Sell" ratings.
*   **WESTERN DIGITAL CORP (WDC)** has the highest proportion of "Buy" ratings among its peers, with 19 "Buy" and 6 "Strong Buy" ratings, and no "Sell" or "Strong Sell" ratings.
*   **HP INC (HPQ)** stands out with the most cautious outlook, having the highest number of "Hold" (16) and "Sell" (6) ratings, and the lowest number of "Strong Buy" ratings (1) among the group.
*   **NETAPP INC (NTAP)** and **HEWLETT PACKARD ENTERPRISE (HPE)** have a higher concentration of "Hold" ratings, suggesting analysts are taking a more neutral stance on these companies.
*   Smaller companies like **IONQ INC (IONQ)**, **QUANTUM COMPUTING INC (QUBT)**, and **COMPOSECURE INC-A (CMPO)** have fewer total recommendations, but generally maintain a positive to neutral outlook.

In [35]:
send_message("""Tell me Amazon's current share price and provide candlestick data for the past month. 
Sort the data in descending order by date. Format the prices consistently as currency. 
Round prices to two decimal places. 
Present the data with multiple columns for display in markdown. 
Discuss and provide details about any patterns you notice in the price data. 
Correlate recent patterns with news over the same date range.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
Api.zero_error: model is now gemini-2.5-flash


Generate quote embedding: 0it [00:00, ?it/s]


send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

Amazon's (AMZN) current share price is $234.17, with a change of $0.95 and a percentage change of 0.4073%. The high price of the day was $235.797, the low price was $231.88, and the open price was $231.88. The previous close price was $233.22. This information is as of Mon Dec 1 11:37:16 2025.

Here is the candlestick data for Amazon (AMZN) for the past month, sorted in descending order by date:

| Date | Open | High | Low | Close | Volume |
|---|---|---|---|---|---|
| Fri Nov 28 00:00:00 2025 | $231.24 | $233.29 | $230.22 | $233.22 | 20,250,425 |
| Wed Nov 26 00:00:00 2025 | $230.74 | $231.75 | $228.77 | $229.16 | 38,497,719 |
| Tue Nov 25 00:00:00 2025 | $226.38 | $230.52 | $223.80 | $229.67 | 39,379,339 |
| Mon Nov 24 00:00:00 2025 | $222.56 | $227.33 | $222.27 | $226.28 | 54,318,223 |
| Fri Nov 21 00:00:00 2025 | $216.34 | $222.21 | $215.18 | $220.69 | 68,490,453 |
| Thu Nov 20 00:00:00 2025 | $227.05 | $227.41 | $216.74 | $217.14 | 50,308,862 |
| Wed Nov 19 00:00:00 2025 | $223.74 | $223.74 | $218.52 | $222.69 | 58,335,353 |
| Tue Nov 18 00:00:00 2025 | $228.10 | $230.20 | $222.42 | $222.55 | 60,608,442 |
| Mon Nov 17 00:00:00 2025 | $233.25 | $234.60 | $229.19 | $232.87 | 59,918,908 |
| Fri Nov 14 00:00:00 2025 | $235.06 | $238.73 | $232.89 | $234.69 | 38,956,619 |
| Thu Nov 13 00:00:00 2025 | $243.05 | $243.75 | $236.50 | $237.58 | 41,401,638 |
| Wed Nov 12 00:00:00 2025 | $250.24 | $250.37 | $243.75 | $244.20 | 31,190,063 |
| Tue Nov 11 00:00:00 2025 | $248.41 | $249.75 | $247.23 | $249.10 | 23,563,960 |
| Mon Nov 10 00:00:00 2025 | $248.34 | $251.75 | $245.59 | $248.40 | 36,476,474 |
| Fri Nov 7 00:00:00 2025 | $242.90 | $244.90 | $238.49 | $244.41 | 46,374,294 |
| Thu Nov 6 00:00:00 2025 | $249.15 | $250.38 | $242.17 | $243.04 | 46,004,201 |
| Wed Nov 5 00:00:00 2025 | $249.03 | $251.00 | $246.16 | $250.20 | 40,610,602 |
| Tue Nov 4 00:00:00 2025 | $250.38 | $257.01 | $248.66 | $249.32 | 51,546,311 |
| Mon Nov 3 00:00:00 2025 | $255.36 | $258.60 | $252.90 | $254.00 | 95,997,714 |

**Price Data Patterns and Correlation with News:**

Looking at the candlestick data for Amazon (AMZN) over the past month, several patterns emerge:

*   **Early November Surge and Mid-Month Dip:** The stock started November strong, reaching a high of $258.60 on Mon Nov 3 2025. This initial surge aligns with news from early November, where Amazon announced a significant $38 billion cloud computing services deal with OpenAI, which was seen as a major positive for its AWS segment and AI growth. Several articles from Nov 3rd and 4th highlight this deal and its positive impact on Amazon's stock.

    However, the price then experienced a noticeable dip, falling to a low of $215.18 on Fri Nov 21 2025. This decline coincides with news around Nov 13th-19th, where concerns about AI investment returns, potential market bubbles, and aggressive capital expenditures by tech giants were widely discussed. Articles from Nov 13th and 14th mention a tech stock selloff and Amazon's stock shedding market value. Additionally, news on Nov 19th highlighted Amazon selling shares of quantum computing and AI stocks (IonQ and AMD), potentially taking profits amid valuation concerns.

*   **Late November Recovery:** Towards the end of November, the stock showed signs of recovery, closing at $233.22 on Fri Nov 28 2025. This recovery aligns with a renewed positive sentiment around Amazon's AI initiatives and its strong position in e-commerce and cloud computing. News from Nov 25th-30th consistently highlights Amazon's strong AWS cloud computing business, its AI growth engine, and its potential to reach a $3 trillion market cap. Several articles also mention Amazon's legal victory in New York and its strong performance during Black Friday.

*   **Volume Fluctuations:** The trading volume was notably high at the beginning of November (95,997,714 on Nov 3rd) during the initial price surge, indicating strong investor interest. Volume remained relatively high during the mid-month dip, suggesting active trading during the period of uncertainty. Towards the end of the month, as the stock recovered, the volume was still substantial, but not as high as the initial surge.

In summary, Amazon's stock performance in November 2025 appears to be heavily influenced by news and sentiment surrounding its AI investments and cloud computing business. Initial excitement over a major OpenAI deal drove the stock up, while broader market concerns about AI valuations and Amazon's capital expenditures led to a mid-month correction. The stock then recovered as positive news about its AI growth engine, e-commerce strength, and legal victories re-emerged.

In [36]:
send_message("What is Apple's ticker overview")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Apple Inc. (AAPL) is a technology company based in Cupertino, CA. Their address is ONE APPLE PARK WAY, CUPERTINO, CA, 95014, and their phone number is (408) 996-1010. Apple was listed on December 12, 1980. They have 166,000 employees and a market cap of $4,120,386,034,050.00. Their primary exchange is XNAS. Apple's description states: "Apple is among the largest companies in the world, with a broad portfolio of hardware and software products targeted at consumers and businesses. Apple's iPhone makes up a majority of the firm sales, and Apple's other products like Mac, iPad, and Watch are designed around the iPhone as the focal point of an expansive software ecosystem. Apple has progressively worked to add new applications, like streaming video, subscription bundles, and augmented reality. The firm designs its own software and semiconductors while working with subcontractors like Foxconn and TSMC to build its products and chips. Slightly less than half of Apple's sales come directly through its flagship stores, with a majority of sales coming indirectly through partnerships and distribution."

In [37]:
send_message("What is Google's stock ticker symbol?")

send_message: get function response
send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Google is listed on the NASDAQ stock exchange under the ticker symbols **GOOGL** (Class A shares) and **GOOG** (Class C shares). These symbols now represent Alphabet Inc., Google's holding company.

Google is also listed on the Frankfurt Stock Exchange under the ticker symbol **GGQ1**.

In [38]:
send_message("What is MGM Studio's stock symbol?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


send_message: updating state
Api.generation_fail.next_model: model is now gemini-2.0-flash-exp
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

I am unable to find the stock symbol for MGM Studios.

In [39]:
send_message("What is MGM Studio's owner company stock symbol?")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response


#### Natural language response

MGM Studios is owned by Amazon, and Amazon's stock symbol is AMZN.


In [40]:
send_message("What is Facebook's stock ticker symbol?")

send_message: get function response


Score wiki search by similarity to topic: 0it [00:00, ?it/s]
Generate wiki embeddings: 0it [00:00, ?it/s]


Api.generation_fail.next_model: model is now gemini-2.0-flash
Api.zero_error: model is now gemini-2.5-flash
send_message: updating state
send_message: got a response


#### Natural language response

Facebook's stock ticker symbol is META.

In [41]:
send_message("""Compare Amazon's bullish versus bearish predictions from Oct 01 2025 until today. 
Include a discussion of recommendation trends, and sentiment analysis of news from the same dates. 
Discuss any patterns or correlations you find.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

From October 1, 2025, until today, December 1, 2025, Amazon (AMZN) has consistently received a strong bullish outlook from analysts, while news sentiment has been more mixed, reflecting both significant growth opportunities and some operational challenges.

**Recommendation Trends (October 1, 2025 - December 1, 2025):**
Analyst recommendations for Amazon have been overwhelmingly positive throughout this period:
*   **October 1, 2025:** 52 "buy" recommendations, 3 "hold," and 23 "strongBuy."
*   **November 1, 2025:** 54 "buy" recommendations, 2 "hold," and 22 "strongBuy."
*   **December 1, 2025:** 52 "buy" recommendations, 5 "hold," and 21 "strongBuy."

Notably, there were no "sell" or "strongSell" recommendations during these months, indicating a consistently bullish sentiment from financial analysts.

**Sentiment Analysis of News (October 1, 2025 - December 1, 2025):**

**Bullish/Positive Sentiment:**
*   **AWS Dominance and AI Leadership:** Numerous articles highlight the accelerating growth of Amazon Web Services (AWS), Amazon's cloud computing division, with reported 20% year-over-year revenue increases. Significant investments in AI infrastructure, custom AI chips (Trainium, Inferentia), and strategic partnerships with companies like OpenAI (including a $38 billion cloud infrastructure deal) and Anthropic are consistently cited as major drivers for future growth and profitability. Amazon's role as a key player in the AI revolution and its potential to reach a $3-4 trillion market capitalization are frequently emphasized.
*   **E-commerce Strength and Efficiency:** Amazon's strong market share in e-commerce, improvements in operational efficiency through robotics and AI-driven logistics, and expanding delivery networks contribute to positive sentiment.
*   **Diversified Business Model:** The company's diverse revenue streams, spanning e-commerce, cloud computing, advertising, and emerging technologies like satellite internet, are seen as providing resilience and multiple avenues for growth.
*   **Analyst Confidence:** Many news pieces report positive analyst ratings, increased price targets (e.g., Wedbush raising to $340), and a general belief that Amazon is an attractive long-term investment.

**Bearish/Negative Sentiment:**
*   **Job Cuts and Restructuring:** Several articles report significant corporate layoffs, including up to 30,000 jobs and 15% of the HR staff. While sometimes framed as strategic restructuring for the AI era and cost-cutting measures, these are also viewed as indicators of potential workforce instability and operational challenges.
*   **High Capital Expenditures:** Concerns are raised about Amazon's aggressive capital expenditures in AI infrastructure (e.g., $125 billion planned investment), with some analysts questioning the immediate return on investment and potential impact on free cash flow.
*   **Competition and Market Share:** While a leader, Amazon faces intense competition in cloud computing (from Microsoft Azure and Google Cloud) and digital advertising (from Google and Meta). Some articles note instances where Amazon's stock has underperformed the broader S&P 500 or other "Magnificent Seven" stocks.
*   **Operational Issues:** A major global AWS outage due to a software bug was reported in late October, leading to negative sentiment regarding service reliability.
*   **Partnership Shifts:** UPS is strategically reducing its shipping volumes with Amazon due to lower margins, indicating a shift in that business relationship.

**Patterns and Correlations:**

1.  **AI as a Central Theme:** AI is the most prominent theme, acting as both a significant bullish catalyst and a source of some bearish concerns. The market is excited about Amazon's AI potential, particularly through AWS, but also watchful of the massive investments required and their impact on profitability and workforce.
2.  **AWS Performance is Key:** The performance and strategic moves of AWS are consistently highlighted as critical to Amazon's overall financial health and investor sentiment. Positive news about AWS directly correlates with bullish sentiment for the company.
3.  **Analyst Optimism vs. News Nuance:** There's a clear divergence between the consistently strong bullish analyst recommendations and the more nuanced, sometimes cautionary, tone of news articles. Analysts appear to be focusing on the long-term strategic advantages and growth potential, while news reports cover both the positive developments and the immediate operational challenges.
4.  **Market Concentration:** Amazon is frequently discussed within the context of the "Magnificent Seven" tech stocks, which have largely driven market performance. This highlights Amazon's systemic importance but also raises broader market concerns about concentration risk and potential overvaluation in the tech sector.

In conclusion, Amazon's bullish predictions are strongly supported by its leadership in cloud computing, aggressive AI investments, and diversified business model. Bearish sentiments are primarily linked to large-scale layoffs and the financial implications of massive AI capital expenditures. The overarching pattern is that the market views Amazon's strategic positioning in AI and cloud as a powerful long-term growth engine, despite short-term operational adjustments and competitive pressures.

In [42]:
send_message("""Compare Google's bullish versus bearish predictions from Oct 01 2025 until today. 
Include a discussion of recommendation trends, and sentiment analysis of news from the same dates. 
Discuss any patterns or correlations you find.""")

send_message: get function response
send_message: updating state
Api.refill_rpm 10
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


Score similarity to query: 100%|██████████| 1/1 [00:00<00:00,  3.53it/s]


send_message: updating state
send_message: got a response
send_message: get function response


Generate grounding embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response


...

send_message: get function response
send_message: generated wrong function arguments
Api.generation_fail.next_model: model is now gemini-2.5-flash-preview-09-2025
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Based on the analysis of analyst recommendations and news sentiment for Alphabet Inc. (GOOGL/GOOG) from **October 1, 2025, until today, December 1, 2025**, the overall prediction for the company is overwhelmingly **bullish**.

The market's confidence is driven almost entirely by the company's successful execution of its Artificial Intelligence (AI) strategy, which has translated into strong financial performance and strategic investments.

---

### 1. Recommendation Trends (Bullish vs. Bearish)

Analyst sentiment for Alphabet Inc. (GOOGL) remained highly bullish throughout the period, with a notable increase in conviction.

| Period | Strong Buy | Buy | Hold | Sell | Strong Sell | Total Bullish (Strong Buy + Buy) |
| :--- | :--- | :--- | :--- | :--- | :--- | :--- |
| **Dec 1, 2025** | 20 | 43 | 10 | 0 | 0 | **63 (86.3%)** |
| **Nov 1, 2025** | 21 | 41 | 12 | 0 | 0 | **62 (83.8%)** |
| **Oct 1, 2025** | 21 | 39 | 13 | 0 | 0 | **60 (82.2%)** |

*   **Bullish Prediction:** The number of analysts recommending a **"Buy"** or **"Strong Buy"** increased from 60 to 63, raising the overall bullish percentage from 82.2% to 86.3%.
*   **Bearish Prediction:** There were **zero "Sell" or "Strong Sell"** recommendations in any month, indicating a complete absence of bearish conviction among major analysts. The slight decrease in total recommendations from November to December is due to a few analysts shifting from a neutral "Hold" rating to a more positive "Buy" rating.

---

### 2. Sentiment Analysis of News (October 1, 2025 – November 28, 2025)

News sentiment during this period was overwhelmingly positive, focusing on AI-driven growth and financial strength.

#### Bullish Drivers (Positive Sentiment)

The primary bullish narrative centers on Alphabet's AI leadership and financial execution:

*   **AI Dominance and Innovation:** The launch and success of the **Gemini 3 AI model** and its integration across Google Search, YouTube, and Google Cloud were consistently highlighted. News articles praised Alphabet's vertically integrated AI stack, including its custom **Tensor Processing Units (TPUs)**, which are seen as a cost-effective challenge to Nvidia's GPU dominance.
*   **Record Financial Performance:** The company reported its **first $100 billion quarterly revenue**, driven by strong growth in Google Services (advertising) and **Google Cloud (34% YoY growth)**. Analysts frequently noted the company's strong cash flow and relatively attractive valuation (low P/E ratio) compared to other "Magnificent Seven" stocks.
*   **Strategic Investor Confidence:** A major bullish catalyst was the disclosure of a **$4-5 billion investment by Warren Buffett's Berkshire Hathaway** in Q3, signaling confidence from a prominent value investor in Alphabet's long-term growth and stability.
*   **Massive Infrastructure Investment:** Alphabet announced significant capital expenditure plans, including a **$40 billion investment in Texas** by 2027 for cloud and AI infrastructure, and a **$6.4 billion expansion in Germany**, reinforcing its commitment to the AI race.

#### Bearish and Neutral Factors

The few negative or cautionary themes were generally overshadowed by the positive AI narrative:

*   **Competitive Threats:** The launch of **OpenAI's ChatGPT Atlas browser** caused a minor, short-term stock drop (approx. 2%), raising concerns about a potential challenge to Google's search and Chrome dominance. However, this was quickly mitigated by news of Google's own AI integration into Chrome.
*   **Regulatory/Legal Issues:** The company was named in a **NYC lawsuit over child social media addiction** (alongside Meta and Snap), and faced ongoing, though less prominent, geopolitical concerns about moving production out of China.
*   **Market Bubble Caution:** Several articles warned of a potential **"AI bubble"** or market overvaluation, but often positioned Alphabet as the "least overvalued" or a "safer" long-term AI play due to its diversified, profitable core business.

---

### 3. Patterns and Correlations

A clear and strong correlation exists between the news sentiment and analyst recommendations:

1.  **AI Success Drives Bullish Consensus:** The consistent stream of positive news regarding Alphabet's AI execution (Gemini, TPUs, Cloud growth) directly correlated with the increase in "Buy" ratings and the complete absence of "Sell" ratings. The market views Alphabet's AI strategy as a successful, revenue-generating endeavor, unlike competitors whose AI spending is sometimes viewed with skepticism (e.g., Meta's stock drop after announcing high AI CapEx).
2.  **Value and Growth Alignment:** The news frequently highlighted Alphabet's dual appeal as both a **high-growth AI leader** and a **value stock** (low P/E ratio). This combination attracted both growth investors (driven by AI) and value investors (like Buffett), creating a powerful, sustained bullish momentum that shrugged off minor competitive and regulatory risks.
3.  **Resilience to Disruption:** The pattern shows that while new competitors (like ChatGPT Atlas) can cause temporary stock dips, the market quickly reverts to focusing on Alphabet's core strengths: its massive cash flow, dominant search market, and profitable cloud business, all of which are being successfully fortified by AI.

In [43]:
send_message("""How is the outlook for Apple based on trends and news sentiment from July 01 2025 until today? 
Perform the same analysis on all peers by sub-industry. Then compare Apple result to it's peers.""")

Api.zero_error: model is now gemini-2.5-flash
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
Api.refill_rpm 10
send_message: got a response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response
send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response


...

send_message: get function response
send_message: get function response


Add chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
send_message: got a response


#### Natural language response

Here's an outlook for Apple and its sub-industry peers based on recommendation trends and news sentiment from July 1, 2025, until today, December 1, 2025:

**Apple (AAPL) Outlook:**

*   **Recommendation Trends:** As of December 1, 2025, Apple has a generally positive outlook from analysts, with 15 "Strong Buy" and 23 "Buy" recommendations, compared to 16 "Hold" and 2 "Sell" recommendations.
*   **News Sentiment (July 1, 2025 - December 1, 2025):**
    *   **Positive:** News highlights Apple's strong iPhone sales, particularly the iPhone 17 series, and the continued growth of its high-margin services segment. Strategic investments in AI (including partnerships with Google for Siri and MP Materials for rare earth magnets) and manufacturing diversification to India are seen as positive moves to mitigate risks and drive future growth. FDA approval for hypertension alerts on Apple Watch also contributes to a positive health tech narrative. Warren Buffett's continued, albeit sometimes trimmed, investment in Apple is also noted positively.
    *   **Neutral:** Several articles discuss Apple's stock trading near 52-week highs, prompting questions about its valuation and the pace of its AI strategy compared to competitors. Warren Buffett's trimming of his stake is viewed as a strategic portfolio adjustment rather than a negative signal about the company itself.
    *   **Negative:** Recurring themes include Apple's perceived lag in AI innovation compared to other tech giants, slower revenue growth in some areas, and ongoing regulatory pressures (antitrust lawsuits in India and the EU, and class-action lawsuits regarding Siri's AI capabilities). Some analysts predict that competitors with stronger AI strategies might surpass Apple's market capitalization.

**Apple's Sub-Industry Peers Outlook:**

Apple's sub-industry peers include companies involved in computer hardware, storage, and quantum computing. Here's a summary of their outlooks:

*   **Dell Technologies (DELL):**
    *   **Recommendation Trends:** Very positive, with 8 "Strong Buy" and 16 "Buy" recommendations.
    *   **News Sentiment:** Highly positive, driven by strong demand for AI servers, record AI server shipments, optimistic fiscal 2026 guidance, and strategic partnerships with Nvidia and Broadcom. Dell is seen as an undervalued AI infrastructure play, though some analysts note potential margin pressures from rising memory costs.
*   **Western Digital (WDC):**
    *   **Recommendation Trends:** Very positive, with 6 "Strong Buy" and 19 "Buy" recommendations.
    *   **News Sentiment:** Highly positive, recognized as a top S&P 500 performer in 2025 with significant stock returns and dividend increases. Strong demand for data storage from AI companies and expansion of its System Integration and Test (SIT) Lab for AI are key drivers.
*   **Hewlett Packard Enterprise (HPE):**
    *   **Recommendation Trends:** More balanced, with 6 "Strong Buy" and 7 "Buy" recommendations, alongside 13 "Hold."
    *   **News Sentiment:** Positive, following the successful $14 billion acquisition of Juniper Networks, which is expected to double its networking business and provide AI networking solutions. Collaborations on 5G and sovereign AI supercomputers are also positive. However, strategic restructuring costs have impacted profit margins.
*   **Pure Storage Inc - Class A (PSTG):**
    *   **Recommendation Trends:** Positive, with 7 "Strong Buy" and 13 "Buy" recommendations.
    *   **News Sentiment:** Positive, outperforming Nvidia in 2025 with strong annual recurring revenue growth and key clients in the AI sector.
*   **HP Inc (HPQ):**
    *   **Recommendation Trends:** More negative/hold, with 1 "Strong Buy" and 1 "Buy" against 16 "Hold" and 6 "Sell."
    *   **News Sentiment:** Mixed. While there are positive developments in AI-powered gaming hardware and sustainable products, the company has faced workforce reductions and missed revenue forecasts.
*   **NetApp Inc (NTAP):**
    *   **Recommendation Trends:** More balanced, leaning towards "Hold," with 3 "Strong Buy" and 10 "Buy" recommendations, and 14 "Hold."
    *   **News Sentiment:** Positive, aligning with Broadcom's quantum-safe SAN switch portfolio and focusing on infrastructure modernization and data protection.
*   **Super Micro Computer Inc (SMCI):**
    *   **Recommendation Trends:** Mixed to positive, with 4 "Strong Buy" and 10 "Buy" recommendations, but also 9 "Hold" and 3 "Sell."
    *   **News Sentiment:** Mixed. Positive sentiment stems from launching Nvidia Blackwell Ultra solutions, strong AI order backlogs, and institutional interest. However, negative sentiment arises from missing revenue expectations, stock declines due to weak guidance, and past fraud allegations.
*   **IonQ Inc (IONQ):**
    *   **Recommendation Trends:** Positive, with 2 "Strong Buy" and 10 "Buy" recommendations.
    *   **News Sentiment:** Highly speculative. While showing strong revenue growth and strategic acquisitions in quantum computing, it faces high valuations, significant net losses, and is considered a risky investment. Amazon notably sold its entire stake.
*   **Quantum Computing Inc (QUBT):**
    *   **Recommendation Trends:** Positive, with 2 "Strong Buy" and 5 "Buy" recommendations.
    *   **News Sentiment:** Highly speculative. Despite strong Q3 earnings and a substantial capital raise, the company has minimal sales, high operating expenses, and an expensive valuation. It's considered a high-risk investment.
*   **CompoSecure Inc-A (CMPO):**
    *   **Recommendation Trends:** Positive, with 2 "Strong Buy" and 8 "Buy" recommendations.
    *   **News Sentiment:** Positive, with plans for a business combination and ongoing management fee generation.

**Comparison of Apple to its Peers:**

*   **AI Leadership:** Many of Apple's peers, particularly Dell, Western Digital, Pure Storage, and Super Micro Computer, are seen as directly benefiting from and leading in the AI infrastructure and data storage boom. Apple, while making AI investments and partnerships, is often perceived as lagging in groundbreaking AI innovation compared to these specialized AI-focused companies.
*   **Growth Drivers:** Apple's growth is primarily driven by its established iPhone ecosystem and expanding services. Its peers are seeing significant growth from the broader AI revolution, data center expansion, and specialized technological advancements in areas like quantum computing and rare earth materials.
*   **Valuation & Risk:** Apple's valuation is a point of discussion, with some analysts questioning if it's justified given its slower AI progress. While some peers like Dell and Pure Storage are seen as undervalued, the quantum computing pure-plays (IonQ, QUBT) carry extremely high valuations and significant speculative risk, making them much riskier investments than Apple.
*   **Analyst Sentiment:** While Apple generally has a positive analyst sentiment, some of its peers, particularly Western Digital and Dell, show even stronger "Buy" recommendations, reflecting their more direct and impactful involvement in the current AI-driven market surge.

**Overall Conclusion:**

Apple's outlook is stable and positive, supported by its strong brand, loyal customer base, and growing services revenue. It is actively working to integrate AI into its ecosystem and diversify its manufacturing. However, it is currently perceived as playing catch-up in the AI race compared to some of its peers who are directly building the foundational AI infrastructure.

Many of Apple's peers, especially those in AI hardware and data storage, are experiencing more explosive growth and stronger positive sentiment due to the immense demand for AI technologies. The quantum computing peers, while highly speculative, represent a high-risk, high-reward segment of the tech industry.

In essence, Apple is a robust and profitable company with a solid future, but its growth trajectory is currently more measured compared to the rapid expansion seen in some of its more specialized, AI-centric peers.

In [44]:
send_message("""What does the recent news say about Apple and the impact of tariffs? From 2025-09-01 up to today. 
Also locate candlestick data for the same dates. 
Discuss in detail any correlations in patterns between the candlestick and news data. Ignore duplicate news entry.""")

send_message: get function response
send_message: updating state
send_message: got a response
send_message: get function response
send_message: get function response
limited 4/min, waiting 7.81881856918335s
limited 4/min, waiting 0s


Upsert chunks embedding: 0it [00:00, ?it/s]


send_message: updating state
Api.refill_rpm 10
send_message: got a response


#### Natural language response

From September 1, 2025, to December 1, 2025, Apple's stock (AAPL) experienced a general upward trend, with some periods of volatility. The news during this period highlighted several factors, including the impact of tariffs, which showed a correlation with the stock's movements.

**Impact of Tariffs:**

*   **Negative Correlation:** News directly related to President Trump's tariff threats and Apple's reported tariff-related cost increases often coincided with periods of stock price decline or stagnation. For instance, on **October 10, 2025**, news broke about "Trump Shocks Markets: VIX Spikes 25%, S&P 500 Eyes Worst Day Since April," explicitly stating that Apple experienced a significant stock price decline due to renewed tariff threats against China. Similarly, on **November 18, 2025**, "The Stock Market Flashes a Warning as Investors Get Bad News About President Trump's Tariffs" reported Apple facing $1.1 billion in tariff-related cost increases, which could have contributed to downward pressure. On **October 26, 2025**, an article titled "Is Apple Going to Be Hit Hard by President Trump's Tariffs?" highlighted potential significant challenges from US-China trade tensions and rare earth element export restrictions.
*   **Positive/Mitigating Correlation:** Conversely, news detailing Apple's strategies to mitigate tariff impacts was often met with positive or neutral sentiment, potentially cushioning negative market reactions. For example, on **October 28, 2025**, "Top 3 Stocks Powering Through Trump’s Tariff Policies" noted that Apple successfully avoided arduous tariffs through strategic U.S. investments, relocated iPhone production, and achieved exemptions. This positive news could have contributed to a stock recovery after earlier tariff-related dips. Furthermore, Apple's strategic investments in rare earth materials, such as the $500 million investment in MP Materials for recycling facility development (reported on **October 31, 2025**, and **September 4, 6, 12, 30, November 2, 15, 17, 19, 26, 29, 30, December 1**), were seen as proactive measures to address supply chain risks stemming from China's export controls, which likely instilled investor confidence. The news on **September 17, 2025**, about Apple investing in Vietnam for supply chain diversification also falls into this category.

**Other Significant News and Candlestick Correlations:**

*   **Early September (around $225-$245):** The stock showed an upward trend, coinciding with several positive news items. Notably, favorable Google antitrust rulings (reported on **September 3, 4, 5, 7, 8, 9, 11, 12**) preserved Apple's significant annual revenue from Google, providing a strong positive impetus. News about Apple's AI future and strategic investments also contributed to this positive sentiment.
*   **Mid-September to Early October (fluctuating in $245-$260):** This period saw mixed news. While some reports indicated an "underwhelming" iPhone 17 launch (**September 12, 14, 15**) and concerns about Apple's AI progress (**September 28, October 8**), positive news about strong iPhone 17 pre-order demand in China (**September 22**) and positive analyst upgrades (**October 2**) helped to balance the sentiment, leading to fluctuating but generally stable prices.
*   **Mid-October to Early November (fluctuating in $255-$275):** This period included the negative tariff news mentioned above, which likely caused some dips. However, positive news about strong iPhone 17 sales (**October 23, 24, 30**) and Apple being a top holding in various ETFs (**October 14, 19, 22, 25, 27, 28, 29, 30, 31, November 2, 3, 4, 5, 8, 10, 11, 12, 14, 17, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, December 1**) indicated continued investor confidence.
*   **Late November to December 1 (upward trend towards $277-$279):** The stock showed a strong upward movement. This aligns with continued positive news regarding strong iPhone sales (**November 25, 28**), strategic partnerships, and Apple being a significant holding in various investment funds. Despite some negative news like new antitrust lawsuits in India and a class-action lawsuit over AI training (**November 27, 14**), the overall positive sentiment from product demand and strategic positioning seemed to drive the stock higher.

**In conclusion:** Apple's stock performance during this period was a complex interplay of various factors. While tariff-related news introduced volatility and sometimes downward pressure, Apple's proactive strategies to mitigate these risks, coupled with strong iPhone sales, favorable legal outcomes (like the Google antitrust case), and ongoing AI investments, generally contributed to a positive overall trend in its stock price. The candlestick data reflects these shifts in market sentiment, with dips often correlating with negative news and rallies aligning with positive developments.